In [1]:
%config IPCompleter.use_jedi = False
%pdb off
%load_ext autoreload
%autoreload 3

# required to enable non-blocking interaction:
%gui qt5

import sys
from copy import deepcopy
from typing import Dict, List, Tuple, Optional, Callable, Union, Any
from typing_extensions import TypeAlias
from neuropy.utils.result_context import IdentifyingContext
from nptyping import NDArray
import neuropy.utils.type_aliases as types
from collections import defaultdict

import numpy as np
import pandas as pd
import re
from pathlib import Path
from datetime import datetime

from neuropy.utils.indexing_helpers import PandasHelpers
from pyphocorehelpers.indexing_helpers import partition_df
# Set the maximum number of columns to display
pd.set_option('display.max_columns', 100)

import IPython
from pyphocorehelpers.programming_helpers import IPythonHelpers
from pyphocorehelpers.notebook_helpers import NotebookCellExecutionLogger
from pyphocorehelpers.assertion_helpers import Assert

# Jupyter-lab enable printing for any line on its own (instead of just the last one in the cell)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

_notebook_path:Path = Path(IPythonHelpers.try_find_notebook_filepath(IPython.extract_module_locals())).resolve() # Finds the path of THIS notebook

# Plotting
# import pylustrator # customization of figures
import matplotlib
import matplotlib as mpl
import matplotlib.pyplot as plt
_bak_rcParams = mpl.rcParams.copy()

matplotlib.use('Qt5Agg')

# Switch to the desired interactivity mode
plt.interactive(True)

import seaborn as sns

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
template: str = 'plotly_dark' # set plotl template
pio.templates.default = template
from pyphocorehelpers.plotting.media_output_helpers import fig_to_clipboard
from pyphocorehelpers.Filesystem.path_helpers import file_uri_from_path, sanitize_filename_for_Windows
from pyphocorehelpers.gui.Jupyter.simple_widgets import fullwidth_path_widget, simple_path_display_widget
from pyphoplacecellanalysis.Pho2D.plotly.Extensions.plotly_helpers import plotly_helper_save_figures, _helper_build_figure, plotly_pre_post_delta_scatter, plot_across_sessions_scatter_results
from pyphocorehelpers.assertion_helpers import Assert

# from ..PendingNotebookCode import plot_across_sessions_scatter_results, plot_histograms, plot_stacked_histograms
from pyphocorehelpers.Filesystem.path_helpers import find_first_extant_path
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import find_csv_files, find_HDF5_files, find_most_recent_files
from pyphoplacecellanalysis.Pho2D.statistics_plotting_helpers import plot_histograms_across_sessions, plot_histograms, plot_stacked_histograms

from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.DirectionalPlacefieldGlobalComputationFunctions import DecoderDecodedEpochsResult
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import load_across_sessions_exported_files, _process_and_load_exported_file, _common_cleanup_operations

from pyphocorehelpers.programming_helpers import metadata_attributes
from pyphocorehelpers.function_helpers import function_attributes

from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import build_session_t_delta, _new_process_csv_files, _old_process_csv_files

debug_print: bool = False
enable_neptune: bool = False

_TODAY_DAY_ONLY_DATE: str = "2024-09-25"
TODAY_DAY_DATE: str = f"{_TODAY_DAY_ONLY_DATE}_Apogee"
# TODAY_DAY_DATE: str = f"{_TODAY_DAY_ONLY_DATE}_GL"
# TODAY_DAY_DATE: str = f"{_TODAY_DAY_ONLY_DATE}_Lab"
# TODAY_DAY_DATE: str = f"{_TODAY_DAY_ONLY_DATE}_rMBP"

print(f'TODAY_DAY_DATE: {TODAY_DAY_DATE}')

types.session_str: TypeAlias = str # a unique session identifier

if enable_neptune:
    import neptune # for logging progress and results
    from neptune.types import File
    from pyphoplacecellanalysis.General.Batch.NeptuneAiHelpers import Neptuner, AutoValueConvertingNeptuneRun, set_environment_variables 

    ## Gets the notebook filepath for Neptune:
    import IPython
    from pyphocorehelpers.programming_helpers import IPythonHelpers
    notebook_filepath: str = IPythonHelpers.try_find_notebook_filepath(IPython.extract_module_locals())
    assert Path(notebook_filepath).resolve().exists(), f"found notebook filepath: '{notebook_filepath}' does not exist"
    # notebook_filepath

    neptune_kwargs = {'project':"commander.pho/PhoDibaLongShortAcrossSessions",
    'api_token':"eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiIxOGIxODU2My1lZTNhLTQ2ZWMtOTkzNS02ZTRmNzM5YmNjNjIifQ=="}
            
    neptuner = Neptuner(project_name=neptune_kwargs['project'], api_token=neptune_kwargs['api_token'])


    if neptuner.run is None:
        neptuner.run = AutoValueConvertingNeptuneRun(project=neptuner.project_name, api_token=neptuner.api_token, dependencies="infer", source_files=[notebook_filepath])
        params = {"TODAY_DAY_DATE": TODAY_DAY_DATE, "run_workstation": "Apogee"}
        neptuner.run["parameters"] = params
        neptuner.outputs = neptuner.run['outputs']
        neptuner.figures = neptuner.outputs['figures']

    neptuner_run: AutoValueConvertingNeptuneRun = neptuner.run
    
    # run = neptune.init_run(source_files=["**/*.dvc"])

    # # Pre-execution dataframe view:
    # run["dataset/global_batch_run_progress_df"].upload(File.as_html(global_batch_run.to_dataframe(expand_context=True, good_only=False))) # "path/to/test_preds.csv"

else:
    # no neptune:
    neptuner = None    
    neptuner_run = None


known_bad_sessions = [IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-02_17-46-44')]
known_bad_session_strs = [str(v.get_description()) for v in known_bad_sessions]
known_bad_session_strs

Automatic pdb calling has been turned OFF
TODAY_DAY_DATE: 2024-09-25_Apogee


['kdiba_pin01_one_11-02_17-46-44']

### csv_files

In [2]:
## Load across session t_delta CSV, which contains the t_delta for each session:

## INPUTS: known_bad_session_strs,

# cuttoff_date = datetime(2024, 8, 29)
cuttoff_date = datetime(2024, 7, 1)
# cuttoff_date = datetime(2024, 5, 18)
# cuttoff_date = None

known_collected_outputs_paths = [Path(v).resolve() for v in ['/Users/pho/data/collected_outputs',
                                                            '/Volumes/SwapSSD/Data/collected_outputs', r"K:/scratch/collected_outputs", '/Users/pho/Dropbox (University of Michigan)/MED-DibaLabDropbox/Data/Pho/Outputs/output/collected_outputs', r'C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs',
                                                            '/home/halechr/FastData/collected_outputs/', '/home/halechr/cloud/turbo/Data/Output/collected_outputs']]
collected_outputs_directory = find_first_extant_path(known_collected_outputs_paths)
assert collected_outputs_directory.exists(), f"collected_outputs_directory: {collected_outputs_directory} does not exist! Is the right computer's config commented out above?"
# fullwidth_path_widget(scripts_output_path, file_name_label='Scripts Output Path:')
print(f'collected_outputs_directory: {collected_outputs_directory}')

# _active_folder_widget = fullwidth_path_widget(collected_outputs_directory)
# display(_active_folder_widget)

# Create a 'figures' subfolder if it doesn't exist
figures_folder: Path = collected_outputs_directory.joinpath('figures').resolve()
figures_folder.mkdir(parents=False, exist_ok=True)
assert figures_folder.exists()
print(f'\tfigures_folder: {file_uri_from_path(figures_folder)}')

# Create an output path for the across session collected results (like the aggregate CSVs built from the individual session CSVs)
across_sessions_output_folder: Path = collected_outputs_directory.joinpath('../across_sessions').resolve()
across_sessions_output_folder.mkdir(parents=False, exist_ok=True)
assert across_sessions_output_folder.exists()
print(f'\tacross_sessions_output_folder: {file_uri_from_path(across_sessions_output_folder)}')

## sessions' t_delta:
t_delta_csv_path = collected_outputs_directory.joinpath('../2024-01-18_GL_t_split_df.csv').resolve() # GL
# t_delta_csv_path = collected_outputs_directory.joinpath('2024-06-11_GL_t_split_df.csv').resolve()

t_delta_df, t_delta_dict, (earliest_delta_aligned_t_start, latest_delta_aligned_t_end) = build_session_t_delta(t_delta_csv_path=t_delta_csv_path)

if neptuner_run is not None:
    _neptuner_run_parameters = dict(cuttoff_date=cuttoff_date, collected_outputs_directory=collected_outputs_directory.as_posix(), figures_folder=figures_folder.as_posix(),
                           across_sessions_output_folder=across_sessions_output_folder.as_posix(), t_delta_csv_path=t_delta_csv_path.as_posix())
    for k, v in _neptuner_run_parameters.items():
        neptuner_run[f'parameters/{k}'] = v
    _neptuner_run_parameters = {} # reset after writing

## Find the files:
csv_files = find_csv_files(collected_outputs_directory)
h5_files = find_HDF5_files(collected_outputs_directory)

csv_sessions, parsed_csv_files_df  = find_most_recent_files(found_session_export_paths=csv_files, cuttoff_date=cuttoff_date)
h5_sessions, parsed_h5_files_df = find_most_recent_files(found_session_export_paths=h5_files)

## OUTPUTS: csv_files, csv_sessions, parsed_csv_files_df
## OUTPUTS: h5_files, h5_sessions, parsed_h5_files_df

_neptuner_run_parameters = dict(csv_files=csv_files, h5_files=h5_files, csv_sessions=csv_sessions, h5_sessions=h5_sessions)

# #TODO 2024-03-02 12:12: - [ ] Could add weighted correlation if there is a dataframe for that and it's computed:
_df_raw_variable_names = ['simple_pf_pearson_merged_df', 'weighted_corr_merged_df']
_df_variables_names = ['laps_weighted_corr_merged_df', 'ripple_weighted_corr_merged_df', 'laps_simple_pf_pearson_merged_df', 'ripple_simple_pf_pearson_merged_df']

# # tbin_values_dict = {'laps': self.laps_decoding_time_bin_size, 'ripple': self.ripple_decoding_time_bin_size}
time_col_name_dict = {'laps': 'lap_start_t', 'ripple': 'ripple_start_t'} ## default should be 't_bin_center'

# fold older files:
# {'laps_marginals_df': 'lap_start_t', 'ripple_marginals_df': 'ripple_start_t', 'laps_time_bin_marginals_df':'t_bin_center', 'ripple_time_bin_marginals_df':'t_bin_center'}

if neptuner_run is not None:
    _neptuner_run_parameters = _neptuner_run_parameters | dict(earliest_delta_aligned_t_start=earliest_delta_aligned_t_start, latest_delta_aligned_t_end=latest_delta_aligned_t_end,
                                     t_delta_df=t_delta_df)
    for k, v in _neptuner_run_parameters.items():
        neptuner_run[f'parsed/{k}'] = v
    _neptuner_run_parameters = {} # reset after writing

all_session_experiment_experience_csv_path = Path("./EXTERNAL/sessions_experiment_datetime_df.csv").resolve()
Assert.path_exists(all_session_experiment_experience_csv_path)
## NEW `parsed_csv_files_df1-based approach 2024-07-11 - 
## INPUTS: parsed_csv_files_df
dict_results, df_results, excluded_or_outdated_files_list = _new_process_csv_files(parsed_csv_files_df=parsed_csv_files_df, t_delta_dict=t_delta_dict, cuttoff_date=cuttoff_date, known_bad_session_strs=known_bad_session_strs, all_session_experiment_experience_csv_path=all_session_experiment_experience_csv_path, debug_print=False) # , known_bad_session_strs=known_bad_session_strs
(final_sessions_loaded_laps_dict, final_sessions_loaded_ripple_dict, final_sessions_loaded_laps_time_bin_dict, final_sessions_loaded_ripple_time_bin_dict, final_sessions_loaded_simple_pearson_laps_dict, final_sessions_loaded_simple_pearson_ripple_dict, final_sessions_loaded_laps_wcorr_dict, final_sessions_loaded_ripple_wcorr_dict, final_sessions_loaded_laps_all_scores_dict, final_sessions_loaded_ripple_all_scores_dict) = dict_results
(all_sessions_laps_df, all_sessions_ripple_df, all_sessions_laps_time_bin_df, all_sessions_ripple_time_bin_df, all_sessions_simple_pearson_laps_df, all_sessions_simple_pearson_ripple_df, all_sessions_wcorr_laps_df, all_sessions_wcorr_ripple_df, all_sessions_all_scores_ripple_df) = df_results

collected_outputs_directory: K:\scratch\collected_outputs
	figures_folder: file:///K:/scratch/collected_outputs/figures
	across_sessions_output_folder: file:///K:/scratch/across_sessions
earliest_delta_aligned_t_start: -2057.2259484970764, latest_delta_aligned_t_end: 1661.8560019930592
ERR: Could not parse filename: "session_h5_files_copydict_2024-09-10_GL"
ERR: Could not parse filename: "session_h5_files_copydict_2024-09-11_GL"
WARN: curr_session_t_delta is None for session_str = "11-02_17-46-44"
File type merged_complete_epoch_stats_df not implemented.
WARN: curr_session_t_delta is None for session_str = "11-02_17-46-44"
File type neuron_replay_stats_df not implemented.
WARN: curr_session_t_delta is None for session_str = "11-02_19-28-0"
File type merged_complete_epoch_stats_df not implemented.
WARN: curr_session_t_delta is None for session_str = "11-02_19-28-0"
File type neuron_replay_stats_df not implemented.
WARN: curr_session_t_delta is None for session_str = "2006-4-09_16-40-54"

In [3]:
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import load_and_apply_session_experience_rank_csv

all_session_experiment_experience_csv_path = Path("./EXTERNAL/sessions_experiment_datetime_df.csv").resolve()
Assert.path_exists(all_session_experiment_experience_csv_path)
sessions_df, (experience_rank_map_dict, experience_orientation_rank_map_dict), _callback_add_df_columns = load_and_apply_session_experience_rank_csv(all_session_experiment_experience_csv_path, session_uid_str_sep='_')
df_results = [_callback_add_df_columns(a_df, session_id_column_name='session_name') for a_df in df_results]
all_sessions_all_scores_ripple_df: pd.DataFrame = _callback_add_df_columns(all_sessions_all_scores_ripple_df, session_id_column_name='session_name')


# all_sessions_all_scores_ripple_df
# all_sessions_all_scores_ripple_df['session_name']
sessions_df

,Unnamed: 0,format_name,animal,exper_name,session_name,path,session_datetime,experience_rank,experience_orientation_rank,session_uid
0,31,kdiba,vvp01,two,2006-4-09_16-40-54,W:/Data/KDIBA/vvp01/two/2006-4-09_16-40-54,2006-04-09 16:40:54,0,0,kdiba|vvp01|two|2006-4-09_16-40-54
1,12,kdiba,vvp01,one,2006-4-09_17-29-30,W:/Data/KDIBA/vvp01/one/2006-4-09_17-29-30,2006-04-09 17:29:30,1,0,kdiba|vvp01|one|2006-4-09_17-29-30
2,13,kdiba,vvp01,one,2006-4-10_12-25-50,W:/Data/KDIBA/vvp01/one/2006-4-10_12-25-50,2006-04-10 12:25:50,2,1,kdiba|vvp01|one|2006-4-10_12-25-50
3,32,kdiba,vvp01,two,2006-4-10_12-58-3,W:/Data/KDIBA/vvp01/two/2006-4-10_12-58-3,2006-04-10 12:58:03,3,1,kdiba|vvp01|two|2006-4-10_12-58-3
4,33,kdiba,vvp01,two,2006-4-10_19-11-57,W:/Data/KDIBA/vvp01/two/2006-4-10_19-11-57,2006-04-10 19:11:57,4,2,kdiba|vvp01|two|2006-4-10_19-11-57
...,...,...,...,...,...,...,...,...,...,...
63,59,kdiba,pin01,one,11-09_21-17-16,W:/Data/KDIBA/pin01/one/11-09_21-17-16,2006-11-09 21:17:16,11,11,kdiba|pin01|one|11-09_21-17-16
64,60,kdiba,pin01,one,11-09_22-4-5,W:/Data/KDIBA/pin01/one/11-09_22-4-5,2006-11-09 22:04:05,12,12,kdiba|pin01|one|11-09_22-4-5
65,61,kdiba,pin01,one,11-19_12-35-59,W:/Data/KDIBA/pin01/one/11-19_12-35-59,2006-11-19 12:35:59,13,13,kdiba|pin01|one|11-19_12-35-59
66,62,kdiba,pin01,one,11-19_13-2-0,W:/Data/KDIBA/pin01/one/11-19_13-2-0,2006-11-19 13:02:00,14,14,kdiba|pin01|one|11-19_13-2-0


In [61]:
# print(list(sessions_df.columns)) # ['Unnamed: 0', 'format_name', 'animal', 'exper_name', 'session_name', 'path', 'session_datetime', 'experience_rank', 'experience_orientation_rank', 'session_uid']

# sessions_df[['format_name', 'animal', 'exper_name', 'session_name']]

# ensure no repeats:
if sessions_df[['format_name', 'animal', 'exper_name', 'session_name']].duplicated().any():
    raise ValueError("Duplicate entries found in the specified columns.")

# sessions_df.drop_duplicates(subset=['format_name', 'animal', 'exper_name', 'session_name'], inplace=False)

if sessions_df[['session_uid']].duplicated().any():
    raise ValueError("Duplicate entries found in the session_uid columns.")


df: pd.DataFrame = deepcopy(sessions_df)

unique_animals = df['animal'].unique().tolist()
unique_exper_names = df['exper_name'].unique().tolist()
unique_session_names = df['session_name'].unique().tolist()

sessions_tree = {}
sessions_number_tree = {}


for animal in unique_animals:
    sessions_tree[animal] = list(unique_session_names)
    sessions_number_tree[animal] = len(list(unique_session_names))
    # unique_exper_names


    # for sess in unique_session_names:
# sessions_tree
sessions_number_tree


{'vvp01': 68, 'gor01': 68, 'pin01': 68}

In [5]:
# Performed 1 aggregation grouped on columns: 'animal', 'exper_name'
sessions_count_df = sessions_df.groupby(['animal', 'exper_name']).agg(session_name_count=('session_name', 'count')).reset_index()
# 	animal	exper_name	session_name_count
# 0	gor01	one	6
# 1	gor01	two	6
# 2	pin01	one	16
# 3	vvp01	one	19
# 4	vvp01	two	21

# sessions_count_df.to_clipboard() #['session_name_count']
sessions_count_df

,animal,exper_name,session_name_count
0,gor01,one,6
1,gor01,two,6
2,pin01,one,16
3,vvp01,one,19
4,vvp01,two,21


In [6]:
all_sessions_all_scores_ripple_df


,start,stop,label,duration,is_valid_epoch,P_LR,P_RL,P_Long,P_Short,P_Long_LR,congruent_dir_bins_ratio_long_LR,coverage_long_LR,direction_change_bin_ratio_long_LR,integral_second_derivative_long_LR,jump_long_LR,longest_sequence_length_ratio_long_LR,pearsonr_long_LR,stddev_of_diff_long_LR,total_congruent_direction_change_long_LR,total_variation_long_LR,travel_long_LR,wcorr_long_LR,P_Long_RL,congruent_dir_bins_ratio_long_RL,coverage_long_RL,direction_change_bin_ratio_long_RL,integral_second_derivative_long_RL,jump_long_RL,longest_sequence_length_ratio_long_RL,pearsonr_long_RL,stddev_of_diff_long_RL,total_congruent_direction_change_long_RL,total_variation_long_RL,travel_long_RL,wcorr_long_RL,P_Short_LR,congruent_dir_bins_ratio_short_LR,coverage_short_LR,direction_change_bin_ratio_short_LR,integral_second_derivative_short_LR,jump_short_LR,longest_sequence_length_ratio_short_LR,pearsonr_short_LR,stddev_of_diff_short_LR,total_congruent_direction_change_short_LR,total_variation_short_LR,travel_short_LR,wcorr_short_LR,P_Short_RL,congruent_dir_bins_ratio_short_RL,...,short_best_score,score_diff,long_best_velocity,short_best_velocity,velocity_diff,long_best_intercept,short_best_intercept,intercept_diff,long_best_speed,short_best_speed,speed_diff,long_best_wcorr,short_best_wcorr,wcorr_diff,long_best_pearsonr,short_best_pearsonr,pearsonr_diff,long_best_travel,short_best_travel,travel_diff,long_best_coverage,short_best_coverage,coverage_diff,long_best_jump,short_best_jump,jump_diff,long_best_longest_sequence_length_ratio,short_best_longest_sequence_length_ratio,longest_sequence_length_ratio_diff,long_best_direction_change_bin_ratio,short_best_direction_change_bin_ratio,direction_change_bin_ratio_diff,long_best_congruent_dir_bins_ratio,short_best_congruent_dir_bins_ratio,congruent_dir_bins_ratio_diff,long_best_total_congruent_direction_change,short_best_total_congruent_direction_change,total_congruent_direction_change_diff,long_best_total_variation,short_best_total_variation,total_variation_diff,long_best_integral_second_derivative,short_best_integral_second_derivative,integral_second_derivative_diff,long_best_stddev_of_diff,short_best_stddev_of_diff,stddev_of_diff_diff,session_experience_rank,session_experience_orientation_rank,is_novel_exposure
0,19.584635,19.710587,1,0.125952,True,0.509376,0.490624,0.861733,0.138267,0.438946,0.500000,0.245614,0.500000,117853.583616,0.003087,0.571429,NaN,94.395864,0.000000,453.221023,0.352976,0.115717,0.422787,0.666667,0.263158,0.166667,71365.465847,0.003254,0.714286,NaN,73.656486,0.000000,294.593665,0.229434,1.476788e-01,0.070430,0.333333,0.280702,0.500000,113246.138989,0.005952,0.571429,NaN,103.563241,226.610512,423.006289,0.489591,0.106943,0.067837,0.666667,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,True
1,28.343394,28.453003,2,0.109609,True,0.554285,0.445715,0.880086,0.119914,0.487819,0.400000,0.263158,0.400000,73505.146015,0.003171,0.666667,-0.038696,67.671901,200.172619,256.825247,0.240024,-0.648294,0.392267,0.400000,0.315789,0.600000,173314.093553,0.003254,0.500000,0.582680,102.602273,0.000000,430.559972,0.402392,-2.344744e-02,0.066467,0.600000,0.210526,0.400000,183627.351960,0.005828,0.666667,-0.167127,105.848646,305.924191,430.559972,0.598000,-0.620315,0.053447,0.400000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,True
2,57.231484,57.443913,3,0.212429,False,0.379173,0.620827,0.479174,0.520826,0.181690,0.166667,0.614035,0.166667,88240.410100,0.003421,0.846154,-0.573668,62.298018,196.395777,351.246293,0.136778,0.056294,0.297484,1.083333,0.526316,0.000000,0.000000,0.000000,1.000000,-0.015647,0.000000,0.000000,0.000000,0.000000,1.122023e-18,0.197484,0.166667,0.754386,0.166667,117668.144668,0.006

In [ ]:
# all_sessions_ripple_time_bin_df

# h5_files

In [7]:
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import load_across_sessions_exported_h5_files

## INPUTS: h5_sessions, session_dict, cuttoff_date, known_bad_session_strs
parsed_h5_files_df, h5_contexts_paths_dict = load_across_sessions_exported_h5_files(collected_outputs_directory=collected_outputs_directory, cuttoff_date=cuttoff_date,
                                                                                    known_bad_session_strs=known_bad_session_strs)
h5_session_contexts = list(h5_contexts_paths_dict.keys())
included_h5_paths = list(h5_contexts_paths_dict.values())

if neptuner_run is not None:
    _neptuner_run_parameters = dict(parsed_h5_files_df=parsed_h5_files_df,
                                       )
    for k, v in _neptuner_run_parameters.items():
        neptuner_run[f'parsed/{k}'] = v
    _neptuner_run_parameters = {} # reset after writing

parsed_h5_files_df
# h5_contexts_paths_dict


collected_outputs_directory: K:\scratch\collected_outputs


,export_datetime,session,custom_replay_name,file_type,decoding_time_bin_size_str,path
2,2024-09-11,11-02_17-46-44,,pipeline_results,,K:\scratch\collected_outputs\2024-09-11_GL_11-...
5,2024-09-11,11-02_17-46-44,,time_bin_size_sweep_results,,K:\scratch\collected_outputs\2024-09-11_GL-11-...
8,2024-09-11,11-02_19-28-0,,pipeline_results,,K:\scratch\collected_outputs\2024-09-11_GL_11-...
11,2024-09-11,11-02_19-28-0,,time_bin_size_sweep_results,,K:\scratch\collected_outputs\2024-09-11_GL-11-...
14,2024-09-11,11-03_12-3-25,,pipeline_results,,K:\scratch\collected_outputs\2024-09-11_GL_11-...
17,2024-09-11,2006-4-09_16-40-54,,pipeline_results,,K:\scratch\collected_outputs\2024-09-11_GL_200...
20,2024-09-11,2006-4-09_16-40-54,,time_bin_size_sweep_results,,K:\scratch\collected_outputs\2024-09-11_GL-200...
23,2024-09-11,2006-4-09_17-29-30,,pipeline_results,,K:\scratch\collected_outputs\2024-09-11_GL_200...
26,2024-09-11,2006-4-09_17-29-30,,time_bin_size_sweep_results,,K:\scratch\collected_outputs\2024-09-11_GL-200...
29,2024-09-11,2006-4-10_12-25-50,,pipeline_results,,K:\scratch\collected_outputs\2024-09-11_GL_200...


In [8]:
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import AcrossSessionTables
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import AcrossSessionsVisualizations
from pyphocorehelpers.indexing_helpers import reorder_columns, reorder_columns_relative

included_session_contexts = deepcopy(h5_session_contexts)
included_h5_paths = deepcopy(included_h5_paths)
num_sessions = len(included_session_contexts)
(neuron_identities_table, long_short_fr_indicies_analysis_table, neuron_replay_stats_table), output_path_dicts = AcrossSessionTables.build_and_save_all_combined_tables(included_session_contexts, included_h5_paths,
                                                                                                                                                    override_output_parent_path=across_sessions_output_folder, output_path_suffix=f'{TODAY_DAY_DATE}',
                                                                                                                                                    should_restore_native_column_types=True, include_csv=True, include_pkl=True)


if neptuner_run is not None:
    _neptuner_run_parameters = dict(neuron_identities_table=neuron_identities_table, long_short_fr_indicies_analysis_table=long_short_fr_indicies_analysis_table, neuron_replay_stats_table=neuron_replay_stats_table,
                                       num_sessions=num_sessions)
    for k, v in _neptuner_run_parameters.items():
        neptuner_run[f'parsed/{k}'] = v
    _neptuner_run_parameters = {} # reset after writing

    for output_name, a_paths_dict in output_path_dicts.items():
        for format_extension, an_output_path in a_paths_dict.items():
            neptuner_run[f"output_files/{format_extension}/{output_name}"].upload(an_output_path.resolve().as_posix())


# {'neuron_identities_table': {'.csv': WindowsPath('K:/scratch/across_sessions/2024-06-05_Apogee/neuron_identities_table.csv'),
#   '.pkl': WindowsPath('K:/scratch/across_sessions/2024-06-05_Apogee/neuron_identities_table.pkl')},
#  'long_short_fr_indicies_analysis_table': {'.csv': WindowsPath('K:/scratch/across_sessions/2024-06-05_Apogee/long_short_fr_indicies_analysis_table.csv'),
#   '.pkl': WindowsPath('K:/scratch/across_sessions/2024-06-05_Apogee/long_short_fr_indicies_analysis_table.pkl')},
#  'neuron_replay_stats_table': {'.csv': WindowsPath('K:/scratch/across_sessions/2024-06-05_Apogee/neuron_replay_stats_table.csv'),
#   '.pkl': WindowsPath('K:/scratch/across_sessions/2024-06-05_Apogee/neuron_replay_stats_table.pkl')}}

output_path_dicts

## Move the "height" columns to the end
neuron_replay_stats_table = reorder_columns_relative(neuron_replay_stats_table, column_names=['neuron_uid', 'format_name', 'animal', 'exper_name', 'session_name', 'neuron_type', 'aclu', 'session_uid', 'session_datetime'], relative_mode='start')

neuron_replay_stats_table


failed for file path: K:\scratch\collected_outputs\2024-09-11_GL_2006-6-12_15-55-31_pipeline_results.h5, table_key: /kdiba/gor01/one/2006-6-12_15-55-31/neuron_identities/table. wth exception group ``/`` does not have a child named ``/kdiba/gor01/one/2006-6-12_15-55-31/neuron_identities/table``. Skipping.
failed for file path: K:\scratch\collected_outputs\2024-09-11_GL_11-03_12-3-25_pipeline_results.h5, table_key: /kdiba/pin01/one/11-03_12-3-25/neuron_identities/table. wth exception group ``/`` does not have a child named ``/kdiba/pin01/one/11-03_12-3-25/neuron_identities/table``. Skipping.
concatenating dataframes from 13 of 15 files
failed for file path: K:\scratch\collected_outputs\2024-09-11_GL_2006-6-12_15-55-31_pipeline_results.h5, table_key: /kdiba/gor01/one/2006-6-12_15-55-31/global_computations/long_short_fr_indicies_analysis/table. wth exception group ``/`` does not have a child named ``/kdiba/gor01/one/2006-6-12_15-55-31/global_computations/long_short_fr_indicies_analysis/tab

{'neuron_identities_table': {'.csv': WindowsPath('K:/scratch/across_sessions/2024-09-25_Apogee/neuron_identities_table.csv'),
  '.pkl': WindowsPath('K:/scratch/across_sessions/2024-09-25_Apogee/neuron_identities_table.pkl')},
 'long_short_fr_indicies_analysis_table': {'.csv': WindowsPath('K:/scratch/across_sessions/2024-09-25_Apogee/long_short_fr_indicies_analysis_table.csv'),
  '.pkl': WindowsPath('K:/scratch/across_sessions/2024-09-25_Apogee/long_short_fr_indicies_analysis_table.pkl')},
 'neuron_replay_stats_table': {'.csv': WindowsPath('K:/scratch/across_sessions/2024-09-25_Apogee/neuron_replay_stats_table.csv'),
  '.pkl': WindowsPath('K:/scratch/across_sessions/2024-09-25_Apogee/neuron_replay_stats_table.pkl')}}

,neuron_uid,format_name,animal,exper_name,session_name,neuron_type,aclu,session_uid,session_datetime,long_pf_peak_x,short_pf_peak_x,track_membership,long_non_replay_mean,short_non_replay_mean,non_replay_diff,long_replay_mean,short_replay_mean,replay_diff,long_mean,short_mean,mean_diff,num_replays,long_num_replays,short_num_replays,custom_frs_index,is_rate_extrema,is_refined_exclusive,is_refined_LxC,is_refined_SxC,is_long_peak_left_cap,is_long_peak_right_cap,is_long_peak_either_cap,LS_pf_peak_x_diff,long_LR_pf2D_peak_x,long_LR_pf2D_peak_y,long_RL_pf2D_peak_x,long_RL_pf2D_peak_y,short_LR_pf2D_peak_x,short_LR_pf2D_peak_y,short_RL_pf2D_peak_x,short_RL_pf2D_peak_y,long_LR_pf1D_peak,long_RL_pf1D_peak,short_LR_pf1D_peak,short_RL_pf1D_peak,peak_diff_LR_pf1D_peak,peak_diff_RL_pf1D_peak,session_experience_rank,session_experience_orientation_rank,is_novel_exposure
0,kdiba|gor01|one|2006-6-08_14-26-15|2,kdiba,gor01,one,2006-6-08_14-26-15,0,2,kdiba|gor01|one|2006-6-08_14-26-15,2006-06-08 14:26:15,NaN,NaN,1,0.412424,0.115533,-0.296891,7.91274,NaN,NaN,4.162582,NaN,NaN,45,45,0,0.233430,False,False,False,False,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,1,False
1,kdiba|gor01|one|2006-6-08_14-26-15|3,kdiba,gor01,one,2006-6-08_14-26-15,0,3,kdiba|gor01|one|2006-6-08_14-26-15,2006-06-08 14:26:15,141.385524,198.280871,1,0.274069,1.394595,1.120526,9.241991,8.536258,-0.705733,4.75803,4.965426,0.207397,91,23,68,-0.692958,True,False,False,False,0.0,0.0,0.0,-56.895346,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,1,False
2,kdiba|gor01|one|2006-6-08_14-26-15|4,kdiba,gor01,one,2006-6-08_14-26-15,0,4,kdiba|gor01|one|2006-6-08_14-26-15,2006-06-08 14:26:15,NaN,179.315755,2,0.119382,0.924896,0.805514,NaN,12.484801,NaN,NaN,6.704848,NaN,87,0,87,-0.982542,True,True,False,True,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,1,False
3,kdiba|gor01|one|2006-6-08_14-26-15|5,kdiba,gor01,one,2006-6-08_14-26-15,0,5,kdiba|gor01|one|2006-6-08_14-26-15,2006-06-08 14:26:15,99.662271,202.073894,1,2.326046,2.012612,-0.313434,11.994904,11.315249,-0.679655,7.160475,6.66393,-0.496545,229,126,103,-0.035629,False,False,False,False,0.0,0.0,0.0,-102.411623,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,1,False
4,kdiba|gor01|one|2006-6-08_14-26-15|6,kdiba,gor01,one,2006-6-08_14-26-15,2,6,kdiba|gor01|one|2006-6-08_14-26-15,2006-06-08 14:26:15,NaN,NaN,1,25.389639,12.754636,-12.635004,105.811996,41.364962,-64.447034,65.600818,27.059799,-38.541019,602,345,257,0.099237,False,False,False,False,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
801,kdiba|pin01|one|fet11-01_12-58-54|28,kdiba,pin01,one,fet11-01_12-58-54,pyr,28,kdiba|pin01|one|fet11-01_12-58-54,2009-11-01 12:58:54,180.571251,176.844138,SHARED,1.227517,1.3169564137781686,0.08943915035797345,8.804706579928968,9.155474207048053,0.35076762711908493,5.0161119216745815,5.236215310413111,0.2201033887385293,181,100,81,-0.087569,False,False,False,False,NaN,NaN,NaN,NaN,175.339442,141.003729,186.412849,140.908855,176.615749,140.846341,142.356312,146.959402,55.712956,59.440069,130.255222,100.438316,74.542266,40.998246,0,0,True
802,kdiba|pin01|one|fet11-01_12-58-54|29,kdiba,pin01,one,fet11-01_12-58-54,pyr,29,kdiba|pin01|one|fet11-01_12-58-54,2009-11-01 12:58:54,57.576513,102.301872,SHARED,0.929991,0.8388929037677681,-0.0910981891459075,8.640241822933044,8.10684461459008,-0.5333972083429632,4.78511645792336,4.472868759178924,-0.3122476987444358,328,233,95,0.019260,False,False,False,False,NaN,NaN,NaN,NaN,81.552880,140.860160,48.385781,147.082681,162.371929,141.001542,82.035449,147.112137,122.800995,115.346769,152.617902,126.528109,29.816906,11.181340,0,0,True
803,kdiba|pin01|one|fet11-01_12-58-54|30,kdiba,pin01,one,fet11-01_12-58-54,

In [9]:
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import AcrossSessionsResults

# output_path_suffix: str = f'2024-09-11_GL'
output_path_suffix: str = f'2024-09-12'
inst_fr_output_filename: str = f'across_session_result_long_short_recomputed_inst_firing_rate_{output_path_suffix}.pkl'
across_session_inst_fr_computation, across_sessions_instantaneous_fr_dict, across_sessions_instantaneous_frs_list = AcrossSessionsResults.load_across_sessions_data(global_data_root_parent_path=collected_outputs_directory, inst_fr_output_filename=inst_fr_output_filename)
# across_sessions_instantaneous_fr_dict = loadData(global_batch_result_inst_fr_file_path)
num_sessions = len(across_sessions_instantaneous_fr_dict)
print(f'num_sessions: {num_sessions}')


global_batch_result_inst_fr_file_path: K:\scratch\collected_outputs\across_session_result_long_short_recomputed_inst_firing_rate_2024-09-12.pkl
Computing loaded session pickle file results : "K:/scratch/collected_outputs/across_session_result_long_short_recomputed_inst_firing_rate_2024-09-12.pkl"... done.
num_sessions: 13
num_sessions: 13


In [10]:
across_session_inst_fr_computation

InstantaneousSpikeRateGroupsComputation(instantaneous_time_bin_size_seconds=0.01, active_identifying_session_ctx=IdentifyingContext<('kdiba', 13)>, LxC_aclus=array(['a0s0_109', 'a0s1_3', 'a0s1_29', 'a0s1_103', 'a0s3_90', 'a0s4_91',
       'a0s4_95', 'a0s5_38', 'a0s5_59', 'a1s1_23', 'a1s2_25', 'a1s3_14',
       'a1s3_30', 'a1s3_32', 'a2s0_8', 'a2s0_27', 'a2s1_27'], dtype='<U32'), SxC_aclus=array(['a0s0_23', 'a0s0_4', 'a0s0_58', 'a0s1_55', 'a0s2_3', 'a0s2_19',
       'a0s3_8', 'a0s5_8', 'a1s0_41', 'a1s1_29', 'a2s0_17'], dtype='<U32'), Fig2_Replay_FR=[SingleBarResult(mean=1.4940262607698784, std=1.0698624645244517, values=array([0.38000416, 1.62111344, 1.54731936, 2.32263415, 0.5556715 ,
       1.56555207, 3.65829217, 0.        , 1.0019871 , 1.91516326,
       0.65361032, 3.5310506 , 1.81648137, 2.2919923 , 2.09351438,
       0.44406025, 0.        ]), LxC_aclus=array(['a0s0_109', 'a0s1_3', 'a0s1_29', 'a0s1_103', 'a0s3_90', 'a0s4_91',
       'a0s4_95', 'a0s5_38', 'a0s5_59', 'a1s1_23', 'a1s

In [ ]:
across_sessions_instantaneous_frs_list

In [11]:
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import AcrossSessionsResults

# matplotlib_configuration_update(is_interactive=True, backend='Qt5Agg')
# output_path_suffix: str = f'{TODAY_DAY_DATE}'
# output_path_suffix: str = f'2024-09-11_GL'
output_path_suffix: str = f'2024-09-12'
print(F'output_path_suffix: {output_path_suffix}')
AcrossSessionsResults.post_compute_all_sessions_processing(global_data_root_parent_path=collected_outputs_directory, output_path_suffix=output_path_suffix, plotting_enabled=False)

output_path_suffix: 2024-09-12
global_batch_result_inst_fr_file_path: K:\scratch\collected_outputs\across_session_result_long_short_recomputed_inst_firing_rate_2024-09-12.pkl
Computing loaded session pickle file results : "K:/scratch/collected_outputs/across_session_result_long_short_recomputed_inst_firing_rate_2024-09-12.pkl"... done.
num_sessions: 13
num_sessions: 13
a_name: neuron_identities_table
loading K:\scratch\collected_outputs\2024-09-12\neuron_identities_table.pkl.
Computing loaded session pickle file results : "K:/scratch/collected_outputs/2024-09-12/neuron_identities_table.pkl"... done.
a_name: long_short_fr_indicies_analysis_table
loading K:\scratch\collected_outputs\2024-09-12\long_short_fr_indicies_analysis_table.pkl.
Computing loaded session pickle file results : "K:/scratch/collected_outputs/2024-09-12/long_short_fr_indicies_analysis_table.pkl"... done.
a_name: neuron_replay_stats_table
loading K:\scratch\collected_outputs\2024-09-12\neuron_replay_stats_table.pkl.
Com

{}

# Get filtered for a particular type of replay:

In [12]:
## filter by specific set of replays:
# dfs_list = (all_sessions_ripple_df, all_sessions_ripple_time_bin_df, all_sessions_simple_pearson_ripple_df, all_sessions_wcorr_ripple_df, all_sessions_all_scores_ripple_df)

replay_name: str = ''
# replay_name: str = 'withNewComputedReplays-qclu_[1, 2]-frateThresh_5.0' # 4307 rows
# replay_name: str = 'withNewKamranExportedReplays-qclu_[1,2]-frateThresh_5.0' # 1417 rows, 1437 rows
# replay_name: str = 'withNormalComputedReplays-qclu_[1,2]-frateThresh_1.0' # 2802 rows, 2831 rows

time_bin_size: float = 0.025
# time_bin_size: float = 0.02
# time_bin_size: float = 0.01
filtered_all_sessions_ripple_df = deepcopy(all_sessions_ripple_df)[(all_sessions_ripple_df['custom_replay_name'] == replay_name) & (all_sessions_ripple_df['time_bin_size'] == time_bin_size)]
filtered_all_sessions_ripple_time_bin_df = deepcopy(all_sessions_ripple_time_bin_df)[(all_sessions_ripple_time_bin_df['custom_replay_name'] == replay_name) & (all_sessions_ripple_time_bin_df['time_bin_size'] == time_bin_size)]
filtered_all_sessions_simple_pearson_ripple_df = deepcopy(all_sessions_simple_pearson_ripple_df)[(all_sessions_simple_pearson_ripple_df['custom_replay_name'] == replay_name) & (all_sessions_simple_pearson_ripple_df['time_bin_size'] == time_bin_size)]
filtered_all_sessions_wcorr_ripple_df = deepcopy(all_sessions_wcorr_ripple_df)[(all_sessions_wcorr_ripple_df['custom_replay_name'] == replay_name) & (all_sessions_wcorr_ripple_df['time_bin_size'] == time_bin_size)]
filtered_all_sessions_all_scores_ripple_df = deepcopy(all_sessions_all_scores_ripple_df)[(all_sessions_all_scores_ripple_df['custom_replay_name'] == replay_name) & (all_sessions_all_scores_ripple_df['time_bin_size'] == time_bin_size)]
## OUTPUTS: filtered_all_sessions_ripple_df, filtered_all_sessions_ripple_time_bin_df, filtered_all_sessions_simple_pearson_ripple_df, filtered_all_sessions_wcorr_ripple_df, filtered_all_sessions_all_scores_ripple_df
# filtered_all_sessions_simple_pearson_ripple_df
filtered_all_sessions_all_scores_ripple_df


,start,stop,label,duration,is_valid_epoch,P_LR,P_RL,P_Long,P_Short,P_Long_LR,congruent_dir_bins_ratio_long_LR,coverage_long_LR,direction_change_bin_ratio_long_LR,integral_second_derivative_long_LR,jump_long_LR,longest_sequence_length_ratio_long_LR,pearsonr_long_LR,stddev_of_diff_long_LR,total_congruent_direction_change_long_LR,total_variation_long_LR,travel_long_LR,wcorr_long_LR,P_Long_RL,congruent_dir_bins_ratio_long_RL,coverage_long_RL,direction_change_bin_ratio_long_RL,integral_second_derivative_long_RL,jump_long_RL,longest_sequence_length_ratio_long_RL,pearsonr_long_RL,stddev_of_diff_long_RL,total_congruent_direction_change_long_RL,total_variation_long_RL,travel_long_RL,wcorr_long_RL,P_Short_LR,congruent_dir_bins_ratio_short_LR,coverage_short_LR,direction_change_bin_ratio_short_LR,integral_second_derivative_short_LR,jump_short_LR,longest_sequence_length_ratio_short_LR,pearsonr_short_LR,stddev_of_diff_short_LR,total_congruent_direction_change_short_LR,total_variation_short_LR,travel_short_LR,wcorr_short_LR,P_Short_RL,congruent_dir_bins_ratio_short_RL,...,short_best_score,score_diff,long_best_velocity,short_best_velocity,velocity_diff,long_best_intercept,short_best_intercept,intercept_diff,long_best_speed,short_best_speed,speed_diff,long_best_wcorr,short_best_wcorr,wcorr_diff,long_best_pearsonr,short_best_pearsonr,pearsonr_diff,long_best_travel,short_best_travel,travel_diff,long_best_coverage,short_best_coverage,coverage_diff,long_best_jump,short_best_jump,jump_diff,long_best_longest_sequence_length_ratio,short_best_longest_sequence_length_ratio,longest_sequence_length_ratio_diff,long_best_direction_change_bin_ratio,short_best_direction_change_bin_ratio,direction_change_bin_ratio_diff,long_best_congruent_dir_bins_ratio,short_best_congruent_dir_bins_ratio,congruent_dir_bins_ratio_diff,long_best_total_congruent_direction_change,short_best_total_congruent_direction_change,total_congruent_direction_change_diff,long_best_total_variation,short_best_total_variation,total_variation_diff,long_best_integral_second_derivative,short_best_integral_second_derivative,integral_second_derivative_diff,long_best_stddev_of_diff,short_best_stddev_of_diff,stddev_of_diff_diff,session_experience_rank,session_experience_orientation_rank,is_novel_exposure
0,19.584635,19.710587,1,0.125952,True,0.509376,0.490624,0.861733,0.138267,0.438946,0.500000,0.245614,0.500000,117853.583616,0.003087,0.571429,NaN,94.395864,0.000000,453.221023,0.352976,0.115717,0.422787,0.666667,0.263158,0.166667,71365.465847,0.003254,0.714286,NaN,73.656486,0.000000,294.593665,0.229434,1.476788e-01,0.070430,0.333333,0.280702,0.500000,113246.138989,0.005952,0.571429,NaN,103.563241,226.610512,423.006289,0.489591,0.106943,0.067837,0.666667,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,True
1,28.343394,28.453003,2,0.109609,True,0.554285,0.445715,0.880086,0.119914,0.487819,0.400000,0.263158,0.400000,73505.146015,0.003171,0.666667,-0.038696,67.671901,200.172619,256.825247,0.240024,-0.648294,0.392267,0.400000,0.315789,0.600000,173314.093553,0.003254,0.500000,0.582680,102.602273,0.000000,430.559972,0.402392,-2.344744e-02,0.066467,0.600000,0.210526,0.400000,183627.351960,0.005828,0.666667,-0.167127,105.848646,305.924191,430.559972,0.598000,-0.620315,0.053447,0.400000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,True
2,57.231484,57.443913,3,0.212429,False,0.379173,0.620827,0.479174,0.520826,0.181690,0.166667,0.614035,0.166667,88240.410100,0.003421,0.846154,-0.573668,62.298018,196.395777,351.246293,0.136778,0.056294,0.297484,1.083333,0.526316,0.000000,0.000000,0.000000,1.000000,-0.015647,0.000000,0.000000,0.000000,0.000000,1.122023e-18,0.197484,0.166667,0.754386,0.166667,117668.144668,0.006

In [66]:
# all_sessions_laps_time_bin_df['time_bin_size'].unique() # [ nan, 0.25]

all_sessions_wcorr_laps_df['time_bin_size'].unique() # [ nan, 0.25]

# ([0.025, 0.01 , 0.05 ]

[0.025, 0.030, 0.044, 0.050, 0.058, 0.072, 0.086, 0.100, 0.250, 1.5]
	   

array([0.025, 0.03 , 0.044, 0.058, 0.072, 0.086, 0.1  , 1.5  , 0.25 ,
       0.05 ])

## Session Enumerations

In [ ]:
dataframes = {
    'all_sessions_ripple_df': all_sessions_ripple_df,
    'all_sessions_ripple_time_bin_df': all_sessions_ripple_time_bin_df,
    'all_sessions_simple_pearson_ripple_df': all_sessions_simple_pearson_ripple_df,
    'all_sessions_wcorr_ripple_df': all_sessions_wcorr_ripple_df,
    'all_sessions_all_scores_ripple_df': all_sessions_all_scores_ripple_df
}

for name, df in dataframes.items():
    unique_time_bin_counts = df.groupby('custom_replay_name')['time_bin_size'].nunique()
    print(f"== '{name}':")
    print(unique_time_bin_counts)


In [ ]:
all_sessions_ripple_df['custom_replay_name'].unique()
all_sessions_ripple_time_bin_df['custom_replay_name'].unique()
all_sessions_simple_pearson_ripple_df['custom_replay_name'].unique()
all_sessions_all_scores_ripple_df['custom_replay_name'].unique() 

# ['', 'withOldestImportedReplays-qclu_XX-frateThresh_0.1',
# 'withNewComputedReplays-qclu_[1, 2]-frateThresh_5.0',
# 'withNewKamranExportedReplays-qclu_[1,2]-frateThresh_5.0',
# 'withNormalComputedReplays-qclu_[1,2]-frateThresh_1.0']


all_sessions_all_scores_ripple_df['custom_replay_name'].unique() ## OH no, only this one is missing values

In [ ]:
all_sessions_ripple_df

In [ ]:
# sess_names, replay_names = all_sessions_ripple_df['session_name'].str.split('__') # , maxsplit=1

for a_split in all_sessions_ripple_df['session_name'].str.split('__'):
	print(f'a_split: {a_split}')

	if len(a_split) > 1:
		print(f'a_split: {a_split}')


In [ ]:
all_sessions_ripple_time_bin_df['custom_replay_name'].unique()
# all_sessions_ripple_df['custom_replay_name'].unique()

In [ ]:
# pd.set_option('display.min_rows', 50)
# pd.set_option('display.max_rows', 50)
# pd.set_option('display.show_dimensions', True)

with pd.option_context('display.min_rows', 50, 'display.show_dimensions', True):
    # max_elements
    display(parsed_csv_files_df)

# parsed_csv_files_df[parsed_csv_files_df['file_type'] == 'ripple_all_scores_merged_df']


In [ ]:
parsed_csv_files_df[parsed_csv_files_df['file_type'] == 'ripple_all_scores_merged_df']

# all_sessions_all_scores_ripple_df


In [ ]:
all_sessions_ripple_time_bin_df

In [ ]:
print(list(np.unique(parsed_csv_files_df.file_type))) # ['laps_marginals_df', 'laps_simple_pf_pearson_merged_df', 'laps_time_bin_marginals_df', 'laps_weighted_corr_merged_df', 'merged_complete_epoch_stats_df', 'ripple_all_scores_merged_df', 'ripple_marginals_df', 'ripple_simple_pf_pearson_merged_df', 'ripple_time_bin_marginals_df', 'ripple_weighted_corr_merged_df']

# Across Session CSV Outputs

In [13]:
display(parsed_csv_files_df)

across_sessions_parsed_csv_files_path = across_sessions_output_folder.joinpath(f'{TODAY_DAY_DATE}_parsed_csv_files_df.csv').resolve()
# parsed_csv_files_df.to_clipboard(excel=True)
parsed_csv_files_df.to_csv(across_sessions_parsed_csv_files_path)
display(fullwidth_path_widget(across_sessions_parsed_csv_files_path, file_name_label='across_sessions_parsed_csv_files_path:'))

,export_datetime,session,custom_replay_name,file_type,decoding_time_bin_size_str,path
0,2024-08-30 00:00:00,11-02_17-46-44,,merged_complete_epoch_stats_df,,K:\scratch\collected_outputs\2024-08-30_GL-11-...
2,2024-09-10 00:00:00,11-02_17-46-44,,neuron_replay_stats_df,,K:\scratch\collected_outputs\2024-09-10_GL-11-...
3,2024-08-30 00:00:00,11-02_19-28-0,,merged_complete_epoch_stats_df,,K:\scratch\collected_outputs\2024-08-30_GL-11-...
5,2024-09-10 00:00:00,11-02_19-28-0,,neuron_replay_stats_df,,K:\scratch\collected_outputs\2024-09-10_GL-11-...
6,2024-08-30 00:00:00,2006-4-09_16-40-54,,merged_complete_epoch_stats_df,,K:\scratch\collected_outputs\2024-08-30_GL-200...
...,...,...,...,...,...,...
2897,2024-09-24 00:00:00,kdiba_vvp01_two_2006-4-28_12-38-13_None,,ripple_time_bin_marginals_df,,K:\scratch\collected_outputs\2024-09-24-kdiba_...
2898,2024-09-24 20:10:00,kdiba_vvp01_two_2006-4-28_12-38-13_None,,ripple_weighted_corr_merged_df,0.01,K:\scratch\collected_outputs\2024-09-24_0810PM...
2899,2024-09-24 20:10:00,kdiba_vvp01_two_2006-4-28_12-38-13_None,,ripple_weighted_corr_merged_df,0.02,K:\scratch\collected_outputs\2024-09-24_0810PM...
2900,2024-09-24 20:10:00,kdiba_vvp01_two_2006-4-28_12-38-13_None,,ripple_weighted_corr_merged_df,0.025,K:\scratch\collected_outputs\2024-09-24_0810PM...


Box(children=(Label(value='across_sessions_parsed_csv_files_path:', layout=Layout(width='auto')), HTML(value="…

In [14]:
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import export_across_session_CSVs

final_across_session_summary_CSVs_output_path = across_sessions_output_folder.resolve()
display(fullwidth_path_widget(final_across_session_summary_CSVs_output_path, file_name_label='final_across_session_summary_CSVs_output_path:'))
final_csv_export_paths = export_across_session_CSVs(final_output_path=final_across_session_summary_CSVs_output_path, TODAY_DAY_DATE=TODAY_DAY_DATE,
                                                    all_sessions_laps_df=all_sessions_laps_df,  all_sessions_ripple_df=all_sessions_ripple_df,  all_sessions_laps_time_bin_df=all_sessions_laps_time_bin_df,  all_sessions_ripple_time_bin_df=all_sessions_ripple_time_bin_df, 
                                                    all_sessions_simple_pearson_laps_df=all_sessions_simple_pearson_laps_df,  all_sessions_simple_pearson_ripple_df=all_sessions_simple_pearson_ripple_df,
                                                    all_sessions_all_scores_ripple_df=all_sessions_all_scores_ripple_df,  all_sessions_all_scores_laps_df=None,
                                                )

# final_csv_export_paths: {'AcrossSession_Laps_per-Epoch': WindowsPath('K:/scratch/across_sessions/2024-06-05_Apogee_AcrossSession_Laps_per-Epoch.csv'),
#  'AcrossSession_Ripple_per-Epoch': WindowsPath('K:/scratch/across_sessions/2024-06-05_Apogee_AcrossSession_Ripple_per-Epoch.csv'),
#  'AcrossSession_Laps_per-TimeBin': WindowsPath('K:/scratch/across_sessions/2024-06-05_Apogee_AcrossSession_Laps_per-TimeBin.csv'),
#  'AcrossSession_Ripple_per-TimeBin': WindowsPath('K:/scratch/across_sessions/2024-06-05_Apogee_AcrossSession_Ripple_per-TimeBin.csv'),
#  'AcrossSession_SimplePearson_Laps_per-Epoch': WindowsPath('K:/scratch/across_sessions/2024-06-05_Apogee_AcrossSession_SimplePearson_Laps_per-Epoch.csv'),
#  'AcrossSession_SimplePearson_Ripple_per-Epoch': WindowsPath('K:/scratch/across_sessions/2024-06-05_Apogee_AcrossSession_SimplePearson_Ripple_per-Epoch.csv'),
#  'AcrossSession_AllScores_Ripple_per-Epoch': WindowsPath('K:/scratch/across_sessions/2024-06-05_Apogee_AcrossSession_AllScores_Ripple_per-Epoch.csv')}

if neptuner_run is not None:
    _neptuner_run_parameters = dict(across_sessions_parsed_csv_files_path=across_sessions_parsed_csv_files_path.as_posix(), final_across_session_summary_CSVs_output_path=final_across_session_summary_CSVs_output_path.as_posix(),
                                       )
    for k, v in _neptuner_run_parameters.items():
        neptuner_run[f'output_files/{k}'] = v
    _neptuner_run_parameters = {} # reset after writing

    for k, v in final_csv_export_paths.items():
        neptuner_run[f"output_files/{k}"].upload(v.resolve().as_posix())
        
    neptuner_run.sync()
    
final_csv_export_paths

Box(children=(Label(value='final_across_session_summary_CSVs_output_path:', layout=Layout(width='auto')), HTML…

{'AcrossSession_Laps_per-Epoch': WindowsPath('K:/scratch/across_sessions/2024-09-25_Apogee_AcrossSession_Laps_per-Epoch.csv'),
 'AcrossSession_Ripple_per-Epoch': WindowsPath('K:/scratch/across_sessions/2024-09-25_Apogee_AcrossSession_Ripple_per-Epoch.csv'),
 'AcrossSession_Laps_per-TimeBin': WindowsPath('K:/scratch/across_sessions/2024-09-25_Apogee_AcrossSession_Laps_per-TimeBin.csv'),
 'AcrossSession_Ripple_per-TimeBin': WindowsPath('K:/scratch/across_sessions/2024-09-25_Apogee_AcrossSession_Ripple_per-TimeBin.csv'),
 'AcrossSession_SimplePearson_Laps_per-Epoch': WindowsPath('K:/scratch/across_sessions/2024-09-25_Apogee_AcrossSession_SimplePearson_Laps_per-Epoch.csv'),
 'AcrossSession_SimplePearson_Ripple_per-Epoch': WindowsPath('K:/scratch/across_sessions/2024-09-25_Apogee_AcrossSession_SimplePearson_Ripple_per-Epoch.csv'),
 'AcrossSession_AllScores_Ripple_per-Epoch': WindowsPath('K:/scratch/across_sessions/2024-09-25_Apogee_AcrossSession_AllScores_Ripple_per-Epoch.csv')}

In [ ]:
print(list(all_sessions_all_scores_ripple_df.columns)) # ['start', 'stop', 'label', 'duration', 'is_user_annotated_epoch', 'is_valid_epoch', 'P_LR', 'P_RL', 'P_Long', 'P_Short', 'P_Long_LR', 'score_long_LR', 'velocity_long_LR', 'intercept_long_LR', 'speed_long_LR', 'wcorr_long_LR', 'pearsonr_long_LR', 'travel_long_LR', 'coverage_long_LR', 'jump_long_LR', 'longest_sequence_length_ratio_long_LR', 'direction_change_bin_ratio_long_LR', 'congruent_dir_bins_ratio_long_LR', 'total_congruent_direction_change_long_LR', 'P_Long_RL', 'score_long_RL', 'velocity_long_RL', 'intercept_long_RL', 'speed_long_RL', 'wcorr_long_RL', 'pearsonr_long_RL', 'travel_long_RL', 'coverage_long_RL', 'jump_long_RL', 'longest_sequence_length_ratio_long_RL', 'direction_change_bin_ratio_long_RL', 'congruent_dir_bins_ratio_long_RL', 'total_congruent_direction_change_long_RL', 'P_Short_LR', 'score_short_LR', 'velocity_short_LR', 'intercept_short_LR', 'speed_short_LR', 'wcorr_short_LR', 'pearsonr_short_LR', 'travel_short_LR', 'coverage_short_LR', 'jump_short_LR', 'longest_sequence_length_ratio_short_LR', 'direction_change_bin_ratio_short_LR', 'congruent_dir_bins_ratio_short_LR', 'total_congruent_direction_change_short_LR', 'P_Short_RL', 'score_short_RL', 'velocity_short_RL', 'intercept_short_RL', 'speed_short_RL', 'wcorr_short_RL', 'pearsonr_short_RL', 'travel_short_RL', 'coverage_short_RL', 'jump_short_RL', 'longest_sequence_length_ratio_short_RL', 'direction_change_bin_ratio_short_RL', 'congruent_dir_bins_ratio_short_RL', 'total_congruent_direction_change_short_RL', 'ripple_start_t', 'long_best_travel', 'short_best_travel', 'travel_diff', 'long_best_coverage', 'short_best_coverage', 'coverage_diff', 'long_best_jump', 'short_best_jump', 'jump_diff', 'long_best_longest_sequence_length_ratio', 'short_best_longest_sequence_length_ratio', 'longest_sequence_length_ratio_diff', 'long_best_direction_change_bin_ratio', 'short_best_direction_change_bin_ratio', 'direction_change_bin_ratio_diff', 'long_best_congruent_dir_bins_ratio', 'short_best_congruent_dir_bins_ratio', 'congruent_dir_bins_ratio_diff', 'long_best_total_congruent_direction_change', 'short_best_total_congruent_direction_change', 'total_congruent_direction_change_diff', 'session_name', 'time_bin_size', 'delta_aligned_start_t']
# ['long_best_total_congruent_direction_change', 'short_best_total_congruent_direction_change']
['longest_sequence_length_ratio_diff', 'direction_change_bin_ratio_diff', 'congruent_dir_bins_ratio_diff', 'total_congruent_direction_change_diff']

In [16]:
all_sessions_simple_pearson_ripple_df # 3138 rows × 24 columns

,P_LR,P_RL,P_Long,P_Short,ripple_idx,ripple_start_t,P_Long_LR,P_Long_RL,P_Short_LR,P_Short_RL,most_likely_decoder_index,start,stop,label,duration,end,long_LR_pf_peak_x_pearsonr,long_RL_pf_peak_x_pearsonr,short_LR_pf_peak_x_pearsonr,short_RL_pf_peak_x_pearsonr,best_decoder_index,session_name,time_bin_size,delta_aligned_start_t,custom_replay_name,is_valid_epoch,is_user_annotated_epoch,epoch_idx,long_best_pf_peak_x_pearsonr,short_best_pf_peak_x_pearsonr,wcorr_long_LR,wcorr_long_RL,wcorr_short_LR,wcorr_short_RL,long_best_wcorr,short_best_wcorr,wcorr_abs_diff,pearsonr_abs_diff,session_experience_rank,session_experience_orientation_rank,is_novel_exposure
0,0.509376,0.490624,0.861733,0.138267,0,19.584635,0.438946,0.422787,0.070430,0.067837,0,19.584635,19.710587,1.0,0.125952,19.710587,NaN,NaN,NaN,NaN,0.0,kdiba_gor01_one_2006-6-07_11-26-53,0.025,-1719.568729,,NaN,NaN,0,NaN,NaN,0.115717,1.476788e-01,0.106943,-2.660144e-04,0.147679,-0.000266,0.147413,NaN,0.0,0.0,True
1,0.554285,0.445715,0.880086,0.119914,1,28.343394,0.487819,0.392267,0.066467,0.053447,0,28.343394,28.453003,2.0,0.109609,28.453003,-0.038696,0.582680,-0.167127,0.552499,1.0,kdiba_gor01_one_2006-6-07_11-26-53,0.025,-1710.809970,,NaN,NaN,1,0.582680,0.552499,-0.648294,-2.344744e-02,-0.620315,-1.069122e-01,-0.023447,-0.106912,-0.083465,0.030181,0.0,0.0,True
2,0.379173,0.620827,0.479174,0.520826,2,57.231484,0.181690,0.297484,0.197484,0.323343,3,57.231484,57.443913,3.0,0.212429,57.443913,-0.573668,-0.015647,-0.545541,0.025296,0.0,kdiba_gor01_one_2006-6-07_11-26-53,0.025,-1681.921880,,NaN,NaN,2,-0.573668,-0.545541,0.056294,1.122023e-18,0.100935,1.645265e-18,0.056294,0.100935,-0.044641,0.028127,0.0,0.0,True
3,0.394597,0.605403,0.812257,0.187743,3,80.191232,0.320514,0.491743,0.074083,0.113660,1,80.191232,80.587704,4.0,0.396472,80.587704,-0.796396,NaN,-0.798072,NaN,1.0,kdiba_gor01_one_2006-6-07_11-26-53,0.025,-1658.962132,,NaN,NaN,3,-0.796396,-0.798072,-0.153652,1.353241e-01,-0.020682,2.416172e-02,-0.153652,-0.020682,0.132970,-0.001675,0.0,0.0,True
4,0.387113,0.612887,0.819695,0.180305,4,81.141524,0.317315,0.502380,0.069798,0.110506,1,81.141524,81.427926,5.0,0.286402,81.427926,-0.609459,0.243758,-0.480347,0.160221,0.0,kdiba_gor01_one_2006-6-07_11-26-53,0.025,-1658.011840,,NaN,NaN,4,-0.609459,-0.480347,0.028884,-3.014099e-01,0.031835,-7.242552e-02,0.028884,0.031835,-0.002951,0.129111,0.0,0.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41141,0.637739,0.362261,0.406584,0.593416,38,437.510637,0.259294,0.147289,0.378445,0.214972,2,437.510637,437.882717,47.0,0.372080,NaN,NaN,NaN,NaN,NaN,0.0,kdiba_vvp01_two_2006-4-28_12-38-13_None,0.025,154.540032,,True,NaN,38,NaN,NaN,0.144075,1.621950e-01,0.096966,1.667449e-01,0.162195,0.166745,-0.004550,NaN,NaN,NaN,False
41142,0.768866,0.231134,0.066658,0.933342,39,448.565223,0.051251,0.015407,0.717616,0.215727,2,448.565223,449.106448,48.0,0.541225,NaN,NaN,NaN,NaN,NaN,0.0,kdiba_vvp01_two_2006-4-28_12-38-13_None,0.025,165.594618,,True,NaN,39,NaN,NaN,-0.036650,-2.219116e-01,-0.036432,1.268247e-01,-0.221912,0.126825,0.095087,NaN,NaN,NaN,False
41143,0.602287,0.397713,0.248361,0.751639,40,458.039819,0.149585,0.098776,0.452703,0.298936,2,458.039819,458.121319,51.0,0.081500,NaN,0.019487,0.104360,0.019487,0.137278,3.0,kdiba_vvp01_two_2006-4-28_12-38-13_None,0.025,175.069215,,True,NaN,40,0.104360,0.137278,-0.143293,3.479273e-01,0.020442,6.861498e-01,0.347927,0.686150,-0.338223,-0.032918,NaN,NaN,False
41144,0.422372,0.577628,0.216143,0.783857,41,462.414591,0.091293,0.124850,0.331080,0.452778,3,462.414591,462.620630,52.0,0.206039,NaN,-0.976007,0.596520,0.837102,0.597999,0.0,kdiba_vvp01_two_2006-4-28_12-38-13_None,0.025,179.443986,,False,NaN,41,-0.976007,0.837102,0.345698,1.319086e-01,0.288839,1.172725e-01,0.345698,0.288839,0.056859,0.138905,NaN,NaN,False


In [17]:
all_sessions_simple_pearson_laps_df # 931 rows × 25 columns

,P_LR,P_RL,P_Long,P_Short,lap_idx,lap_start_t,P_Long_LR,P_Long_RL,P_Short_LR,P_Short_RL,most_likely_decoder_index,start,stop,label,duration,lap_id,lap_dir,long_LR_pf_peak_x_pearsonr,long_RL_pf_peak_x_pearsonr,short_LR_pf_peak_x_pearsonr,short_RL_pf_peak_x_pearsonr,best_decoder_index,session_name,time_bin_size,delta_aligned_start_t,custom_replay_name,maze_id,is_LR_dir,is_most_likely_track_identity_Long,is_most_likely_direction_LR,epoch_idx,long_best_pf_peak_x_pearsonr,short_best_pf_peak_x_pearsonr,wcorr_long_LR,wcorr_long_RL,wcorr_short_LR,wcorr_short_RL,long_best_wcorr,short_best_wcorr,wcorr_abs_diff,pearsonr_abs_diff,session_experience_rank,session_experience_orientation_rank,is_novel_exposure
0,0.466100,0.533900,5.404279e-01,0.459572,0,5.250426,2.518934e-01,2.885345e-01,0.214207,0.245366,1,5.250426,13.692337,0,8.441911,1,1,0.473012,-0.272680,0.351855,-0.544966,3,kdiba_gor01_one_2006-6-07_11-26-53,0.025,-1733.902938,,NaN,NaN,NaN,NaN,0,0.473012,0.351855,0.128830,-0.031509,0.058217,0.008237,0.128830,0.058217,0.070613,0.121157,0.0,0.0,True
1,0.376810,0.623190,5.765410e-01,0.423459,1,29.308670,2.172465e-01,3.592945e-01,0.159564,0.263895,1,29.308670,35.914986,1,6.606316,2,0,-0.477886,0.103281,-0.617921,-0.212387,2,kdiba_gor01_one_2006-6-07_11-26-53,0.025,-1709.844694,,NaN,NaN,NaN,NaN,1,-0.477886,-0.617921,-0.008464,0.013793,0.002362,-0.003495,-0.008464,0.002362,0.006101,-0.140035,0.0,0.0,True
2,0.529244,0.470756,6.404247e-01,0.359575,2,64.076829,3.389411e-01,3.014836e-01,0.190303,0.169272,0,64.076829,68.281973,2,4.205144,3,1,0.677903,-0.296874,0.557521,0.147724,0,kdiba_gor01_one_2006-6-07_11-26-53,0.025,-1675.076535,,NaN,NaN,NaN,NaN,2,-0.296874,0.147724,0.078668,-0.017804,-0.009230,0.014523,-0.017804,0.014523,0.003281,0.149151,0.0,0.0,True
3,0.358672,0.641328,6.488038e-01,0.351196,3,105.519279,2.327077e-01,4.160961e-01,0.125964,0.225232,1,105.519279,109.388817,3,3.869538,4,0,-0.372157,0.388497,0.038310,-0.094784,1,kdiba_gor01_one_2006-6-07_11-26-53,0.025,-1633.634085,,NaN,NaN,NaN,NaN,3,-0.372157,0.038310,-0.069497,0.000453,0.002745,0.006392,-0.069497,0.002745,0.066751,0.333847,0.0,0.0,True
4,0.604020,0.395980,6.741865e-01,0.325814,4,127.074109,4.072221e-01,2.669644e-01,0.196798,0.129016,0,127.074109,134.515034,4,7.440925,5,1,0.683839,-0.093265,0.711045,-0.040914,2,kdiba_gor01_one_2006-6-07_11-26-53,0.025,-1612.079255,,NaN,NaN,NaN,NaN,4,-0.093265,-0.040914,0.085781,0.027607,0.021801,0.001702,0.027607,0.001702,0.025905,0.052351,0.0,0.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9622,NaN,NaN,NaN,NaN,79,492.297752,NaN,NaN,NaN,NaN,0,492.297752,495.335331,79,3.037579,80,0,0.076337,-0.562835,-0.526973,-0.830214,3,kdiba_vvp01_two_2006-4-28_12-38-13_None,1.500,209.327147,,NaN,NaN,NaN,NaN,79,-0.562835,-0.830214,-0.007060,-0.386929,-0.443352,-0.981208,-0.386929,-0.981208,-0.594279,-0.267380,NaN,NaN,False
9623,1.000000,0.000000,1.996995e-46,1.000000,80,497.002526,1.996995e-46,0.000000e+00,1.000000,0.000000,2,497.002526,503.842279,80,6.839753,81,1,0.439159,0.323124,0.640771,0.174394,2,kdiba_vvp01_two_2006-4-28_12-38-13_None,1.500,214.031921,,NaN,NaN,NaN,NaN,80,0.323124,0.174394,0.505080,0.023352,0.943587,0.210732,0.023352,0.210732,-0.187379,0.148730,NaN,NaN,False
9624,0.500000,0.500000,4.995963e-01,0.500404,81,507.146030,2.497982e-01,2.497982e-01,0.250202,0.250202,3,507.146030,510.348224,81,3.202194,82,0,0.034472,-0.355392,-0.409640,-0.736704,3,kdiba_vvp01_two_2006-4-28_12-38-13_None,1.500,224.175425,,NaN,NaN,NaN,NaN,81,0.034472,-0.409640,0.064471,-0.449949,-0.216335,-0.989459,0.064471,-0.216335,-0.151864,-0.375169,NaN,NaN,False
9625,1.000000,0.000000,3.177038e-24,1.000000,82,511.952190,3.177038e-24,0.000000e+00,1.000000,0.000000,2,511.952190,515.787403,82,3.835213,83,1,0.629688,0.373851,0.838673,0.396841,2,kdiba_vvp01_two_2006-4-28_12-38-13_None,1.500,228.981585,,NaN,NaN,NaN,NaN,82,0.373851,0.396841,0.9

In [15]:
all_sessions_ripple_df # 17592 rows × 10 columns

,P_LR,P_RL,P_Long,P_Short,ripple_idx,ripple_start_t,session_name,delta_aligned_start_t,time_bin_size,custom_replay_name,epoch_idx,is_valid_epoch,is_user_annotated_epoch,session_experience_rank,session_experience_orientation_rank,is_novel_exposure
0,0.273265,0.726735,0.820598,0.179402,0,19.584635,kdiba_gor01_one_2006-6-07_11-26-53,-1719.568729,NaN,,0,True,False,0.0,0.0,True
1,0.617909,0.382091,0.929092,0.070908,1,28.343394,kdiba_gor01_one_2006-6-07_11-26-53,-1710.809970,NaN,,1,True,False,0.0,0.0,True
2,0.341645,0.658355,0.483155,0.516845,2,57.231484,kdiba_gor01_one_2006-6-07_11-26-53,-1681.921880,NaN,,2,True,False,0.0,0.0,True
3,0.409888,0.590112,0.741177,0.258823,3,80.191232,kdiba_gor01_one_2006-6-07_11-26-53,-1658.962132,NaN,,3,True,False,0.0,0.0,True
4,0.395680,0.604320,0.865712,0.134288,4,81.141524,kdiba_gor01_one_2006-6-07_11-26-53,-1658.011840,NaN,,4,True,False,0.0,0.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40291,0.637739,0.362261,0.406584,0.593416,38,437.510637,kdiba_vvp01_two_2006-4-28_12-38-13_None,154.540032,NaN,,38,True,False,NaN,NaN,False
40292,0.768866,0.231134,0.066658,0.933342,39,448.565223,kdiba_vvp01_two_2006-4-28_12-38-13_None,165.594618,NaN,,39,True,False,NaN,NaN,False
40293,0.602287,0.397713,0.248361,0.751639,40,458.039819,kdiba_vvp01_two_2006-4-28_12-38-13_None,175.069215,NaN,,40,True,False,NaN,NaN,False
40294,0.422372,0.577628,0.216143,0.783857,41,462.414591,kdiba_vvp01_two_2006-4-28_12-38-13_None,179.443986,NaN,,41,True,False,NaN,NaN,False


In [ ]:
csv_files

In [18]:

all_sessions_simple_pearson_ripple_df

,P_LR,P_RL,P_Long,P_Short,ripple_idx,ripple_start_t,P_Long_LR,P_Long_RL,P_Short_LR,P_Short_RL,most_likely_decoder_index,start,stop,label,duration,end,long_LR_pf_peak_x_pearsonr,long_RL_pf_peak_x_pearsonr,short_LR_pf_peak_x_pearsonr,short_RL_pf_peak_x_pearsonr,best_decoder_index,session_name,time_bin_size,delta_aligned_start_t,custom_replay_name,is_valid_epoch,is_user_annotated_epoch,epoch_idx,long_best_pf_peak_x_pearsonr,short_best_pf_peak_x_pearsonr,wcorr_long_LR,wcorr_long_RL,wcorr_short_LR,wcorr_short_RL,long_best_wcorr,short_best_wcorr,wcorr_abs_diff,pearsonr_abs_diff,session_experience_rank,session_experience_orientation_rank,is_novel_exposure
0,0.509376,0.490624,0.861733,0.138267,0,19.584635,0.438946,0.422787,0.070430,0.067837,0,19.584635,19.710587,1.0,0.125952,19.710587,NaN,NaN,NaN,NaN,0.0,kdiba_gor01_one_2006-6-07_11-26-53,0.025,-1719.568729,,NaN,NaN,0,NaN,NaN,0.115717,1.476788e-01,0.106943,-2.660144e-04,0.147679,-0.000266,0.147413,NaN,0.0,0.0,True
1,0.554285,0.445715,0.880086,0.119914,1,28.343394,0.487819,0.392267,0.066467,0.053447,0,28.343394,28.453003,2.0,0.109609,28.453003,-0.038696,0.582680,-0.167127,0.552499,1.0,kdiba_gor01_one_2006-6-07_11-26-53,0.025,-1710.809970,,NaN,NaN,1,0.582680,0.552499,-0.648294,-2.344744e-02,-0.620315,-1.069122e-01,-0.023447,-0.106912,-0.083465,0.030181,0.0,0.0,True
2,0.379173,0.620827,0.479174,0.520826,2,57.231484,0.181690,0.297484,0.197484,0.323343,3,57.231484,57.443913,3.0,0.212429,57.443913,-0.573668,-0.015647,-0.545541,0.025296,0.0,kdiba_gor01_one_2006-6-07_11-26-53,0.025,-1681.921880,,NaN,NaN,2,-0.573668,-0.545541,0.056294,1.122023e-18,0.100935,1.645265e-18,0.056294,0.100935,-0.044641,0.028127,0.0,0.0,True
3,0.394597,0.605403,0.812257,0.187743,3,80.191232,0.320514,0.491743,0.074083,0.113660,1,80.191232,80.587704,4.0,0.396472,80.587704,-0.796396,NaN,-0.798072,NaN,1.0,kdiba_gor01_one_2006-6-07_11-26-53,0.025,-1658.962132,,NaN,NaN,3,-0.796396,-0.798072,-0.153652,1.353241e-01,-0.020682,2.416172e-02,-0.153652,-0.020682,0.132970,-0.001675,0.0,0.0,True
4,0.387113,0.612887,0.819695,0.180305,4,81.141524,0.317315,0.502380,0.069798,0.110506,1,81.141524,81.427926,5.0,0.286402,81.427926,-0.609459,0.243758,-0.480347,0.160221,0.0,kdiba_gor01_one_2006-6-07_11-26-53,0.025,-1658.011840,,NaN,NaN,4,-0.609459,-0.480347,0.028884,-3.014099e-01,0.031835,-7.242552e-02,0.028884,0.031835,-0.002951,0.129111,0.0,0.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41141,0.637739,0.362261,0.406584,0.593416,38,437.510637,0.259294,0.147289,0.378445,0.214972,2,437.510637,437.882717,47.0,0.372080,NaN,NaN,NaN,NaN,NaN,0.0,kdiba_vvp01_two_2006-4-28_12-38-13_None,0.025,154.540032,,True,NaN,38,NaN,NaN,0.144075,1.621950e-01,0.096966,1.667449e-01,0.162195,0.166745,-0.004550,NaN,NaN,NaN,False
41142,0.768866,0.231134,0.066658,0.933342,39,448.565223,0.051251,0.015407,0.717616,0.215727,2,448.565223,449.106448,48.0,0.541225,NaN,NaN,NaN,NaN,NaN,0.0,kdiba_vvp01_two_2006-4-28_12-38-13_None,0.025,165.594618,,True,NaN,39,NaN,NaN,-0.036650,-2.219116e-01,-0.036432,1.268247e-01,-0.221912,0.126825,0.095087,NaN,NaN,NaN,False
41143,0.602287,0.397713,0.248361,0.751639,40,458.039819,0.149585,0.098776,0.452703,0.298936,2,458.039819,458.121319,51.0,0.081500,NaN,0.019487,0.104360,0.019487,0.137278,3.0,kdiba_vvp01_two_2006-4-28_12-38-13_None,0.025,175.069215,,True,NaN,40,0.104360,0.137278,-0.143293,3.479273e-01,0.020442,6.861498e-01,0.347927,0.686150,-0.338223,-0.032918,NaN,NaN,False
41144,0.422372,0.577628,0.216143,0.783857,41,462.414591,0.091293,0.124850,0.331080,0.452778,3,462.414591,462.620630,52.0,0.206039,NaN,-0.976007,0.596520,0.837102,0.597999,0.0,kdiba_vvp01_two_2006-4-28_12-38-13_None,0.025,179.443986,,False,NaN,41,-0.976007,0.837102,0.345698,1.319086e-01,0.288839,1.172725e-01,0.345698,0.288839,0.056859,0.138905,NaN,NaN,False


# 2024-03-02 - Get only the user-annotated ripples:

In [ ]:
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import _split_user_annotated_ripple_df

## Bump
# input_df = all_sessions_simple_pearson_ripple_df
# input_df = all_sessions_all_scores_ripple_df

all_sessions_all_scores_ripple_df, (valid_ripple_df, invalid_ripple_df), (user_approved_ripple_df, user_rejected_ripple_df) = _split_user_annotated_ripple_df(all_sessions_all_scores_ripple_df)

## 2024-03-14 - 'is_valid_epoch' column
# 'is_valid_epoch'
## OUTPUTS: valid_ripple_df, invalid_ripple_df, user_approved_ripple_df, user_rejected_ripple_df, (user_annotated_epoch_unique_session_names, unannotated_session_names)
user_approved_ripple_df

### 2024-02-29 - 4pm - Filter the events for those meeting wcorr criteria:


In [ ]:
# df: pd.DataFrame = deepcopy(all_sessions_user_annotated_ripple_df)
df: pd.DataFrame = deepcopy(valid_ripple_df) # valid epochs, but not just those that the user approved
# df: pd.DataFrame = deepcopy(user_approved_ripple_df)

## INPUTS: df

min_wcorr_threshold: float = 0.33
min_wcorr_diff_threshold: float = 0.2

# is_included_large_wcorr_diff = np.any((df[['wcorr_abs_diff']].abs() > min_wcorr_diff_threshold), axis=1)
is_included_large_wcorr_diff = np.any((df[['wcorr_diff']].abs() > min_wcorr_diff_threshold), axis=1)
is_included_high_wcorr = np.any((df[['long_best_wcorr', 'short_best_wcorr']].abs() > min_wcorr_threshold), axis=1)

df = df[is_included_high_wcorr]
df

# wcorr_long_LR


In [ ]:
all_sessions_all_scores_ripple_df.time_bin_size.unique() # does not seem to return NaNs

In [ ]:
all_sessions_ripple_time_bin_df.time_bin_size.unique()

In [ ]:
all_sessions_ripple_time_bin_df

### Single-time bin size version:

In [22]:
from pyphoplacecellanalysis.SpecificResults.PendingNotebookCode import build_single_time_bin_size_dfs

# Select the periods:
# active_all_sessions_all_scores_ripple_df, (active_all_sessions_ripple_df, active_all_sessions_ripple_time_bin_df) = all_sessions_all_scores_ripple_df, (all_sessions_ripple_df, all_sessions_ripple_time_bin_df)
active_all_sessions_all_scores_ripple_df, (active_all_sessions_ripple_df, active_all_sessions_ripple_time_bin_df) = filtered_all_sessions_all_scores_ripple_df, (filtered_all_sessions_ripple_df, filtered_all_sessions_ripple_time_bin_df)

# target_time_bin_size: float = np.nan
# target_time_bin_size: float = 0.005 # 0.025 # 0.08222222
# target_time_bin_size: float = 0.02
target_time_bin_size: float = 0.025

## INPUTS: all_sessions_all_scores_ripple_df, (all_sessions_ripple_df, all_sessions_ripple_time_bin_df)
single_time_bin_size_all_sessions_ripple_df, single_time_bin_size_all_sessions_ripple_time_bin_df = build_single_time_bin_size_dfs(all_sessions_all_scores_epochs_df=active_all_sessions_all_scores_ripple_df,
                                                                                                            all_sessions_epochs_df=active_all_sessions_ripple_df, all_sessions_epochs_time_bin_df=active_all_sessions_ripple_time_bin_df, target_time_bin_size=target_time_bin_size,
                                                                                                            included_columns = ['delta_aligned_start_t', 'is_user_annotated_epoch', 'is_valid_epoch'])

                                                                                                            
single_time_bin_size_all_sessions_laps_df, single_time_bin_size_all_sessions_laps_time_bin_df = build_single_time_bin_size_dfs(all_sessions_all_scores_epochs_df=all_sessions_simple_pearson_laps_df,
                                                                                                            all_sessions_epochs_df=all_sessions_laps_df, all_sessions_epochs_time_bin_df=all_sessions_laps_time_bin_df, target_time_bin_size=target_time_bin_size,
                                                                                                            included_columns = ['delta_aligned_start_t',])

## OUTPUTS: single_time_bin_size_all_sessions_ripple_df, single_time_bin_size_all_sessions_ripple_time_bin_df
## OUTPUTS: single_time_bin_size_all_sessions_laps_df, single_time_bin_size_all_sessions_laps_time_bin_df 

single_time_bin_size_all_sessions_laps_df


# # Recover the important columns from the newer `all_sessions_all_scores_ripple_df`
# included_columns = ['delta_aligned_start_t', 'is_user_annotated_epoch', 'is_valid_epoch']  # Added 'delta_aligned_start_t' for the merge
# single_time_bin_size_all_sessions_ripple_df = pd.merge(single_time_bin_size_all_sessions_ripple_df, 
#                      all_sessions_all_scores_ripple_df[included_columns], 
#                      on='delta_aligned_start_t', 
#                      how='left')


# single_time_bin_size_all_sessions_ripple_df
## OUTPUTS: single_time_bin_size_all_sessions_ripple_df, single_time_bin_size_all_sessions_ripple_df
# single_time_bin_size_all_sessions_ripple_time_bin_df
all_sessions_simple_pearson_laps_df
single_time_bin_size_all_sessions_laps_time_bin_df

all_sessions_ripple_df.time_bin_size.unique(): [0.025]
np.shape(single_time_bin_size_all_sessions_ripple_df): (4770, 17)
all_sessions_ripple_time_bin_df.time_bin_size.unique(): [0.025]
np.shape(single_time_bin_size_all_sessions_ripple_time_bin_df): (38186, 14)
all_sessions_ripple_df.time_bin_size.unique(): [ nan 0.25]
np.shape(single_time_bin_size_all_sessions_ripple_df): (0, 15)
all_sessions_ripple_time_bin_df.time_bin_size.unique(): [ nan 0.25]
np.shape(single_time_bin_size_all_sessions_ripple_time_bin_df): (0, 14)


C:\Users\pho\repos\Spike3DWorkEnv\pyPhoPlaceCellAnalysis\src\pyphoplacecellanalysis\SpecificResults\PendingNotebookCode.py:585: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  a_df['pre_post_delta_category'][a_df['delta_aligned_start_t'] < 0.0] = 'pre-delta'
C:\Users\pho\repos\Spike3DWorkEnv\pyPhoPlaceCellAnalysis\src\pyphoplacecellanalysis\SpecificResults\PendingNotebookCode.py:585: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  a_df['pre_post_delta_category'][a_df['delta_aligned_start_t'] < 0.0] = 'pre-delta'
C:\Users\pho\repos\Spike3DWorkEnv\pyPhoPlaceCellAnalysis\src\pyphoplacecellanalysis\SpecificResults\PendingNoteb

,P_LR,P_RL,P_Long,P_Short,lap_idx,lap_start_t,session_name,time_bin_size,custom_replay_name,epoch_idx,session_experience_rank,session_experience_orientation_rank,is_novel_exposure,pre_post_delta_category,delta_aligned_start_t


,P_LR,P_RL,P_Long,P_Short,lap_idx,lap_start_t,P_Long_LR,P_Long_RL,P_Short_LR,P_Short_RL,most_likely_decoder_index,start,stop,label,duration,lap_id,lap_dir,long_LR_pf_peak_x_pearsonr,long_RL_pf_peak_x_pearsonr,short_LR_pf_peak_x_pearsonr,short_RL_pf_peak_x_pearsonr,best_decoder_index,session_name,time_bin_size,delta_aligned_start_t,custom_replay_name,maze_id,is_LR_dir,is_most_likely_track_identity_Long,is_most_likely_direction_LR,epoch_idx,long_best_pf_peak_x_pearsonr,short_best_pf_peak_x_pearsonr,wcorr_long_LR,wcorr_long_RL,wcorr_short_LR,wcorr_short_RL,long_best_wcorr,short_best_wcorr,wcorr_abs_diff,pearsonr_abs_diff,session_experience_rank,session_experience_orientation_rank,is_novel_exposure,pre_post_delta_category
0,0.466100,0.533900,5.404279e-01,0.459572,0,5.250426,2.518934e-01,2.885345e-01,0.214207,0.245366,1,5.250426,13.692337,0,8.441911,1,1,0.473012,-0.272680,0.351855,-0.544966,3,kdiba_gor01_one_2006-6-07_11-26-53,0.025,-1733.902938,,NaN,NaN,NaN,NaN,0,0.473012,0.351855,0.128830,-0.031509,0.058217,0.008237,0.128830,0.058217,0.070613,0.121157,0.0,0.0,True,pre-delta
1,0.376810,0.623190,5.765410e-01,0.423459,1,29.308670,2.172465e-01,3.592945e-01,0.159564,0.263895,1,29.308670,35.914986,1,6.606316,2,0,-0.477886,0.103281,-0.617921,-0.212387,2,kdiba_gor01_one_2006-6-07_11-26-53,0.025,-1709.844694,,NaN,NaN,NaN,NaN,1,-0.477886,-0.617921,-0.008464,0.013793,0.002362,-0.003495,-0.008464,0.002362,0.006101,-0.140035,0.0,0.0,True,pre-delta
2,0.529244,0.470756,6.404247e-01,0.359575,2,64.076829,3.389411e-01,3.014836e-01,0.190303,0.169272,0,64.076829,68.281973,2,4.205144,3,1,0.677903,-0.296874,0.557521,0.147724,0,kdiba_gor01_one_2006-6-07_11-26-53,0.025,-1675.076535,,NaN,NaN,NaN,NaN,2,-0.296874,0.147724,0.078668,-0.017804,-0.009230,0.014523,-0.017804,0.014523,0.003281,0.149151,0.0,0.0,True,pre-delta
3,0.358672,0.641328,6.488038e-01,0.351196,3,105.519279,2.327077e-01,4.160961e-01,0.125964,0.225232,1,105.519279,109.388817,3,3.869538,4,0,-0.372157,0.388497,0.038310,-0.094784,1,kdiba_gor01_one_2006-6-07_11-26-53,0.025,-1633.634085,,NaN,NaN,NaN,NaN,3,-0.372157,0.038310,-0.069497,0.000453,0.002745,0.006392,-0.069497,0.002745,0.066751,0.333847,0.0,0.0,True,pre-delta
4,0.604020,0.395980,6.741865e-01,0.325814,4,127.074109,4.072221e-01,2.669644e-01,0.196798,0.129016,0,127.074109,134.515034,4,7.440925,5,1,0.683839,-0.093265,0.711045,-0.040914,2,kdiba_gor01_one_2006-6-07_11-26-53,0.025,-1612.079255,,NaN,NaN,NaN,NaN,4,-0.093265,-0.040914,0.085781,0.027607,0.021801,0.001702,0.027607,0.001702,0.025905,0.052351,0.0,0.0,True,pre-delta
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9622,NaN,NaN,NaN,NaN,79,492.297752,NaN,NaN,NaN,NaN,0,492.297752,495.335331,79,3.037579,80,0,0.076337,-0.562835,-0.526973,-0.830214,3,kdiba_vvp01_two_2006-4-28_12-38-13_None,1.500,209.327147,,NaN,NaN,NaN,NaN,79,-0.562835,-0.830214,-0.007060,-0.386929,-0.443352,-0.981208,-0.386929,-0.981208,-0.594279,-0.267380,NaN,NaN,False,post-delta
9623,1.000000,0.000000,1.996995e-46,1.000000,80,497.002526,1.996995e-46,0.000000e+00,1.000000,0.000000,2,497.002526,503.842279,80,6.839753,81,1,0.439159,0.323124,0.640771,0.174394,2,kdiba_vvp01_two_2006-4-28_12-38-13_None,1.500,214.031921,,NaN,NaN,NaN,NaN,80,0.323124,0.174394,0.505080,0.023352,0.943587,0.210732,0.023352,0.210732,-0.187379,0.148730,NaN,NaN,False,post-delta
9624,0.500000,0.500000,4.995963e-01,0.500404,81,507.146030,2.497982e-01,2.497982e-01,0.250202,0.250202,3,507.146030,510.348224,81,3.202194,82,0,0.034472,-0.355392,-0.409640,-0.736704,3,kdiba_vvp01_two_2006-4-28_12-38-13_None,1.500,224.175425,,NaN,NaN,NaN,NaN,81,0.034472,-0.409640,0.064471,-0.449949,-0.216335,-0.989459,0.064471,-0.216335,-0.151864,-0.375169,NaN,NaN,False,post-delta
9625,1.000000,0.000000,3.177038e-24,1.000000,82,511.952190,3.177038e-24,0.000000e+00,1.000000,0.000000,2,511.952190,515.787403,82,3.835213,83,1,0.629688,0.373851,0.83867

,P_LR,P_RL,P_Long,P_Short,epoch_idx,t_bin_center,session_name,time_bin_size,custom_replay_name,session_experience_rank,session_experience_orientation_rank,is_novel_exposure,pre_post_delta_category,delta_aligned_start_t


In [23]:
# INPUTS: all_sessions_all_scores_ripple_df, (all_sessions_ripple_df, all_sessions_ripple_time_bin_df)
all_sessions_ripple_df

,P_LR,P_RL,P_Long,P_Short,ripple_idx,ripple_start_t,session_name,delta_aligned_start_t,time_bin_size,custom_replay_name,epoch_idx,is_valid_epoch,is_user_annotated_epoch,session_experience_rank,session_experience_orientation_rank,is_novel_exposure
0,0.273265,0.726735,0.820598,0.179402,0,19.584635,kdiba_gor01_one_2006-6-07_11-26-53,-1719.568729,NaN,,0,True,False,0.0,0.0,True
1,0.617909,0.382091,0.929092,0.070908,1,28.343394,kdiba_gor01_one_2006-6-07_11-26-53,-1710.809970,NaN,,1,True,False,0.0,0.0,True
2,0.341645,0.658355,0.483155,0.516845,2,57.231484,kdiba_gor01_one_2006-6-07_11-26-53,-1681.921880,NaN,,2,True,False,0.0,0.0,True
3,0.409888,0.590112,0.741177,0.258823,3,80.191232,kdiba_gor01_one_2006-6-07_11-26-53,-1658.962132,NaN,,3,True,False,0.0,0.0,True
4,0.395680,0.604320,0.865712,0.134288,4,81.141524,kdiba_gor01_one_2006-6-07_11-26-53,-1658.011840,NaN,,4,True,False,0.0,0.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40291,0.637739,0.362261,0.406584,0.593416,38,437.510637,kdiba_vvp01_two_2006-4-28_12-38-13_None,154.540032,NaN,,38,True,False,NaN,NaN,False
40292,0.768866,0.231134,0.066658,0.933342,39,448.565223,kdiba_vvp01_two_2006-4-28_12-38-13_None,165.594618,NaN,,39,True,False,NaN,NaN,False
40293,0.602287,0.397713,0.248361,0.751639,40,458.039819,kdiba_vvp01_two_2006-4-28_12-38-13_None,175.069215,NaN,,40,True,False,NaN,NaN,False
40294,0.422372,0.577628,0.216143,0.783857,41,462.414591,kdiba_vvp01_two_2006-4-28_12-38-13_None,179.443986,NaN,,41,True,False,NaN,NaN,False


In [24]:
all_sessions_ripple_time_bin_df

,P_LR,P_RL,P_Long,P_Short,epoch_idx,t_bin_center,session_name,delta_aligned_start_t,time_bin_size,custom_replay_name,session_experience_rank,session_experience_orientation_rank,is_novel_exposure
0,5.508644e-01,0.449136,0.993271,0.006729,0,19.599635,kdiba_gor01_one_2006-6-07_11-26-53,-1719.553729,NaN,,0.0,0.0,True
1,4.154438e-01,0.584556,0.289122,0.710878,0,19.629635,kdiba_gor01_one_2006-6-07_11-26-53,-1719.523729,NaN,,0.0,0.0,True
2,1.250169e-01,0.874983,1.000000,0.000000,0,19.659635,kdiba_gor01_one_2006-6-07_11-26-53,-1719.493729,NaN,,0.0,0.0,True
3,1.733444e-03,0.998267,1.000000,0.000000,0,19.689635,kdiba_gor01_one_2006-6-07_11-26-53,-1719.463729,NaN,,0.0,0.0,True
4,6.728078e-01,0.327192,0.920368,0.079632,1,28.358394,kdiba_gor01_one_2006-6-07_11-26-53,-1710.794970,NaN,,0.0,0.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
298655,1.311971e-07,1.000000,0.160933,0.839067,42,492.161234,kdiba_vvp01_two_2006-4-28_12-38-13_None,209.190629,NaN,,NaN,NaN,False
298656,1.303388e-06,0.999999,0.346035,0.653965,42,492.186234,kdiba_vvp01_two_2006-4-28_12-38-13_None,209.215629,NaN,,NaN,NaN,False
298657,5.023392e-01,0.497661,0.396160,0.603840,42,492.211234,kdiba_vvp01_two_2006-4-28_12-38-13_None,209.240629,NaN,,NaN,NaN,False
298658,5.901099e-01,0.409890,0.508888,0.491112,42,492.236234,kdiba_vvp01_two_2006-4-28_12-38-13_None,209.265629,NaN,,NaN,NaN,False


# 🟢🔜 2024-03-28 - AcrossSessionTable (PhoDibaPaper2023 formats) .h5 and figure exports

neuron_identities_table, long_short_fr_indicies_analysis_table, neuron_replay_stats_table: all tables of the same length, one entry per neuron

In [25]:
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import AcrossSessionTables, AcrossSessionsResults, AcrossSessionsVisualizations
from neuropy.utils.mixins.HDF5_representable import HDF_Converter
from pyphoplacecellanalysis.General.Batch.runBatch import BatchResultDataframeAccessor

output_path_suffix: str = '2024-09-12'
# inst_fr_output_filename: str = f'across_session_result_long_short_recomputed_inst_firing_rate_{output_path_suffix}.pkl'
# inst_fr_output_filename: str = f'across_session_result_long_short_recomputed_inst_firing_rate_{output_path_suffix}_0.0009.pkl' # single time bin size
# inst_fr_output_filename: str = f'across_session_result_long_short_recomputed_inst_firing_rate_{output_path_suffix}_0.0015.pkl' # single time bin size
# inst_fr_output_filename: str = f'across_session_result_long_short_recomputed_inst_firing_rate_{output_path_suffix}_0.0025.pkl' # single time bin size
# inst_fr_output_filename: str = f'across_session_result_long_short_recomputed_inst_firing_rate_{output_path_suffix}_0.025.pkl' # single time bin size
inst_fr_output_filename: str = f'across_session_result_long_short_recomputed_inst_firing_rate_{output_path_suffix}_1000.0.pkl' # single time bin size

## INPUTS: included_session_contexts, included_h5_paths
neuron_identities_table, long_short_fr_indicies_analysis_table, neuron_replay_stats_table = AcrossSessionTables.build_all_known_tables(included_session_contexts, included_h5_paths, should_restore_native_column_types=True)

## different than load_all_combined_tables, which seems to work with `long_short_fr_indicies_analysis_table`
# graphics_output_dict |= AcrossSessionsVisualizations.across_sessions_firing_rate_index_figure(long_short_fr_indicies_analysis_results=long_short_fr_indicies_analysis_table, num_sessions=num_sessions, save_figure=True)

## Load all across-session tables from the pickles:
output_path_suffix: str = f'{output_path_suffix}'
neuron_identities_table, long_short_fr_indicies_analysis_table, neuron_replay_stats_table = AcrossSessionTables.load_all_combined_tables(override_output_parent_path=collected_outputs_directory, output_path_suffix=output_path_suffix) # output_path_suffix=f'2023-10-04-GL-Recomp'
# num_sessions = len(neuron_replay_stats_table.session_uid.unique().to_numpy())
# print(f'num_sessions: {num_sessions}')
num_sessions: int = len(long_short_fr_indicies_analysis_table['session_uid'].unique())
print(f'num_sessions: {num_sessions}')

inst_fr_output_load_filepath: Path = collected_outputs_directory.joinpath(inst_fr_output_filename).resolve() # single time bin size # non-instantaneous version
assert inst_fr_output_load_filepath.exists()
# inst_fr_output_filename: str = inst_fr_output_load_filepath.name
# across_session_inst_fr_computation, across_sessions_instantaneous_fr_dict, across_sessions_instantaneous_frs_list = AcrossSessionsResults.load_across_sessions_data(global_data_root_parent_path=global_data_root_parent_path, inst_fr_output_filename=inst_fr_output_filename)
across_session_inst_fr_computation, across_sessions_instantaneous_fr_dict, across_sessions_instantaneous_frs_list = AcrossSessionsResults.load_across_sessions_data(global_data_root_parent_path=inst_fr_output_load_filepath.parent, inst_fr_output_filename=inst_fr_output_filename)

graphics_output_dict = AcrossSessionsResults.post_compute_all_sessions_processing(global_data_root_parent_path=collected_outputs_directory, output_path_suffix=output_path_suffix, plotting_enabled=True, output_override_path=Path('../../output'), inst_fr_output_filename=inst_fr_output_filename)

num_sessions = len(across_sessions_instantaneous_fr_dict)
print(f'num_sessions: {num_sessions}')

# Convert byte strings to regular strings
neuron_replay_stats_table = neuron_replay_stats_table.applymap(lambda x: x.decode('utf-8') if isinstance(x, bytes) else x)
neuron_replay_stats_table

failed for file path: K:\scratch\collected_outputs\2024-09-11_GL_2006-6-12_15-55-31_pipeline_results.h5, table_key: /kdiba/gor01/one/2006-6-12_15-55-31/neuron_identities/table. wth exception group ``/`` does not have a child named ``/kdiba/gor01/one/2006-6-12_15-55-31/neuron_identities/table``. Skipping.
failed for file path: K:\scratch\collected_outputs\2024-09-11_GL_11-03_12-3-25_pipeline_results.h5, table_key: /kdiba/pin01/one/11-03_12-3-25/neuron_identities/table. wth exception group ``/`` does not have a child named ``/kdiba/pin01/one/11-03_12-3-25/neuron_identities/table``. Skipping.
concatenating dataframes from 13 of 15 files
failed for file path: K:\scratch\collected_outputs\2024-09-11_GL_2006-6-12_15-55-31_pipeline_results.h5, table_key: /kdiba/gor01/one/2006-6-12_15-55-31/global_computations/long_short_fr_indicies_analysis/table. wth exception group ``/`` does not have a child named ``/kdiba/gor01/one/2006-6-12_15-55-31/global_computations/long_short_fr_indicies_analysis/tab

,format_name,animal,exper_name,session_name,long_pf_peak_x,short_pf_peak_x,track_membership,long_non_replay_mean,short_non_replay_mean,non_replay_diff,long_replay_mean,short_replay_mean,replay_diff,long_mean,short_mean,mean_diff,num_replays,long_num_replays,short_num_replays,neuron_type,aclu,custom_frs_index,is_rate_extrema,is_refined_exclusive,is_refined_LxC,is_refined_SxC,is_long_peak_left_cap,is_long_peak_right_cap,is_long_peak_either_cap,LS_pf_peak_x_diff,session_uid,neuron_uid,long_LR_pf2D_peak_x,long_LR_pf2D_peak_y,long_RL_pf2D_peak_x,long_RL_pf2D_peak_y,short_LR_pf2D_peak_x,short_LR_pf2D_peak_y,short_RL_pf2D_peak_x,short_RL_pf2D_peak_y,long_LR_pf1D_peak,long_RL_pf1D_peak,short_LR_pf1D_peak,short_RL_pf1D_peak,peak_diff_LR_pf1D_peak,peak_diff_RL_pf1D_peak,session_datetime,session_experience_rank,session_experience_orientation_rank,is_novel_exposure
0,kdiba,gor01,one,2006-6-08_14-26-15,NaN,NaN,1,0.412424,0.115533,-0.296891,7.91274,NaN,NaN,4.162582,NaN,NaN,45,45,0,0,2,0.233430,False,False,False,False,0.0,0.0,0.0,NaN,kdiba|gor01|one|2006-6-08_14-26-15,kdiba|gor01|one|2006-6-08_14-26-15|2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2006-06-08 14:26:15,2,1,False
1,kdiba,gor01,one,2006-6-08_14-26-15,141.385524,198.280871,1,0.274069,1.394595,1.120526,9.241991,8.536258,-0.705733,4.75803,4.965426,0.207397,91,23,68,0,3,-0.692958,True,False,False,False,0.0,0.0,0.0,-56.895346,kdiba|gor01|one|2006-6-08_14-26-15,kdiba|gor01|one|2006-6-08_14-26-15|3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2006-06-08 14:26:15,2,1,False
2,kdiba,gor01,one,2006-6-08_14-26-15,NaN,179.315755,2,0.119382,0.924896,0.805514,NaN,12.484801,NaN,NaN,6.704848,NaN,87,0,87,0,4,-0.982542,True,True,False,True,0.0,0.0,0.0,NaN,kdiba|gor01|one|2006-6-08_14-26-15,kdiba|gor01|one|2006-6-08_14-26-15|4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2006-06-08 14:26:15,2,1,False
3,kdiba,gor01,one,2006-6-08_14-26-15,99.662271,202.073894,1,2.326046,2.012612,-0.313434,11.994904,11.315249,-0.679655,7.160475,6.66393,-0.496545,229,126,103,0,5,-0.035629,False,False,False,False,0.0,0.0,0.0,-102.411623,kdiba|gor01|one|2006-6-08_14-26-15,kdiba|gor01|one|2006-6-08_14-26-15|5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2006-06-08 14:26:15,2,1,False
4,kdiba,gor01,one,2006-6-08_14-26-15,NaN,NaN,1,25.389639,12.754636,-12.635004,105.811996,41.364962,-64.447034,65.600818,27.059799,-38.541019,602,345,257,2,6,0.099237,False,False,False,False,0.0,0.0,0.0,NaN,kdiba|gor01|one|2006-6-08_14-26-15,kdiba|gor01|one|2006-6-08_14-26-15|6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2006-06-08 14:26:15,2,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
801,kdiba,pin01,one,fet11-01_12-58-54,180.571251,176.844138,SHARED,1.227517,1.3169564137781686,0.08943915035797345,8.804706579928968,9.155474207048053,0.35076762711908493,5.0161119216745815,5.236215310413111,0.2201033887385293,181,100,81,pyr,28,-0.087569,False,False,False,False,NaN,NaN,NaN,NaN,kdiba|pin01|one|fet11-01_12-58-54,kdiba|pin01|one|fet11-01_12-58-54|28,175.339442,141.003729,186.412849,140.908855,176.615749,140.846341,142.356312,146.959402,55.712956,59.440069,130.255222,100.438316,74.542266,40.998246,2009-11-01 12:58:54,0,0,True
802,kdiba,pin01,one,fet11-01_12-58-54,57.576513,102.301872,SHARED,0.929991,0.8388929037677681,-0.0910981891459075,8.640241822933044,8.10684461459008,-0.5333972083429632,4.78511645792336,4.472868759178924,-0.3122476987444358,328,233,95,pyr,29,0.019260,False,False,False,False,NaN,NaN,NaN,NaN,kdiba|pin01|one|fet11-01_12-58-54,kdiba|pin01|one|fet11-01_12-58-54|29,81.552880,140.860160,48.385781,147.082681,162.371929,141.001542,82.035449,147.112137,122.800995,115.346769,152.617902,126.528109,29.816906,11.181340,2009-11-01 12:58:54,0,0,True
803,kdiba,pin01,one,fet11-01_12-58-54,120.937439,128.391665,SHARED,0.952187

### Figure Generation

In [26]:
import matplotlib
from neuropy.core.user_annotations import UserAnnotationsManager
from pyphocorehelpers.DataStructure.RenderPlots.MatplotLibRenderPlots import MatplotlibRenderPlots
from neuropy.utils.matplotlib_helpers import matplotlib_configuration_update
# matplotlib.use('Qt5Agg')
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import AcrossSessionsResults, AcrossSessionTables, AcrossSessionsVisualizations
from pyphoplacecellanalysis.SpecificResults.PhoDiba2023Paper import PaperFigureTwo

def _save_matplotlib_fig(matplotlib_output_container: MatplotlibRenderPlots):

    a_fig_context = matplotlib_output_container.context
    assert len(matplotlib_output_container.saved_figures) == 1
    a_saved_fig_path = matplotlib_output_container.saved_figures[0][0]
    assert a_saved_fig_path.exists()

    if neptuner_run is not None:
        a_full_figure_path_key: str = a_fig_context.get_description(separator='/', include_property_names=True, key_value_separator=':') # .replace(' ', '_')
        # a_full_figure_path_key: str = a_fig_context.get_description(separator=':', include_property_names=True, key_value_separator='|')
        print(f'a_full_figure_path_key: "{a_full_figure_path_key}"')
        # neptuner_run['outputs']['figures'][f"{a_full_figure_path_key}"].upload(a_fig)
        neptuner_run['outputs']['figures'][f"{a_full_figure_path_key}"].upload(a_saved_fig_path.as_posix())
        # neptuner.figures[f"{a_full_figure_path_key}"].upload(a_fig)
        

%matplotlib inline


# with matplotlib_configuration_update(is_interactive=False, backend='nbAgg'):
# 'collected_outputs/across_session_result_long_short_recomputed_inst_firing_rate_2024-06-11_GL.pkl'





In [ ]:
long_short_fr_indicies_analysis_table

In [ ]:
neuron_identities_table

In [ ]:
neuron_replay_stats_table

In [27]:
from pyphoplacecellanalysis.SpecificResults.PhoDiba2023Paper import pho_stats_bar_graph_t_tests

LxC_Laps_T_result, SxC_Laps_T_result, LxC_Replay_T_result, SxC_Replay_T_result = pho_stats_bar_graph_t_tests(across_session_inst_fr_computation)

# LxC_Laps_T_result: TtestResult(statistic=5.550057784140024, pvalue=4.394229331160663e-05, df=16)
# SxC_Laps_T_result: TtestResult(statistic=-4.50982955925142, pvalue=0.001125880142367611, df=10)
# LxC_Replay_T_result: TtestResult(statistic=-0.4086778656072959, pvalue=0.6881937588138113, df=16)
# SxC_Replay_T_result: TtestResult(statistic=-3.551930809035679, pvalue=0.0052513298000637825, df=10)



# LxC_Replay_T_result is NOT p<0.05 significant (pvalue=0.6882)
# SxC_Replay_T_result IS NOT p<0.05 significant (pvalue=0.0052513298000637825)

LxC_Laps_T_result: TtestResult(statistic=5.550057784140024, pvalue=4.394229331160663e-05, df=16)
SxC_Laps_T_result: TtestResult(statistic=-4.50982955925142, pvalue=0.001125880142367611, df=10)
LxC_Replay_T_result: TtestResult(statistic=-0.4086778656072959, pvalue=0.6881937588138113, df=16)
SxC_Replay_T_result: TtestResult(statistic=-3.551930809035679, pvalue=0.0052513298000637825, df=10)


In [28]:
from pyphoplacecellanalysis.SpecificResults.PhoDiba2023Paper import PaperFigureTwo
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import AcrossSessionsVisualizations

# %matplotlib inline 
%matplotlib qt5
## Plotting:
graphics_output_dict = {}

# matplotlib_configuration_update(is_interactive=True, backend='Qt5Agg')

## 2023-10-04 - Run `AcrossSessionsVisualizations` corresponding to the PhoDibaPaper2023 figures for all sessions
## Hacks the `PaperFigureTwo` and `InstantaneousSpikeRateGroupsComputation`
global_multi_session_context, _out_aggregate_fig_2 = AcrossSessionsVisualizations.across_sessions_bar_graphs(across_session_inst_fr_computation, num_sessions=num_sessions, enable_tiny_point_labels=False, enable_hover_labels=False, enabled_point_connection_lines=True, save_figure=False)
# plt.show()

num_sessions: 13
a_label: a0s0_109, a_point_list: [(8.525476073623084e-05, 0.0028000000000000004), (0.9115305790315383, 0.00022500000000000002)]
a_label: a0s1_3, a_point_list: [(0.1063119012761338, 0.008022727272727277), (0.8797821189340479, 0.0007750000000000002)]
a_label: a0s1_29, a_point_list: [(0.03511310464978701, 0.005704545454545456), (0.9624121087841876, 0.0008750000000000002)]
a_label: a0s1_103, a_point_list: [(0.14031280296145346, 0.009204545454545458), (0.9583490543239813, 0.0008500000000000001)]
a_label: a0s3_90, a_point_list: [(-0.004785298916425373, 0.002736842105263158), (1.010845884167372, 5e-05)]
a_label: a0s4_91, a_point_list: [(0.09683651965794435, 0.0061063829787234075), (0.8665909424298718, 0.0003658536585365854)]
a_label: a0s4_95, a_point_list: [(0.005397517410685171, 0.021680851063829786), (1.0972940378627822, 0.0003414634146341464)]
a_label: a0s5_38, a_point_list: [(0.07111982437966932, 0.0011750000000000003), (0.97158419988887, 2.5e-05)]
a_label: a0s5_59, a_poi

In [ ]:
# across_session_inst_fr_computation
## Document `InstantaneousSpikeRateGroupsComputation`
from pyphocorehelpers.print_helpers import DocumentationFilePrinter
from pyphocorehelpers.print_helpers import print_keys_if_possible

doc_output_parent_folder = Path(r'C:\Users\pho\repos\Spike3DWorkEnv\Spike3D\EXTERNAL\DEVELOPER_NOTES\DataStructureDocumentation').resolve()
Assert.path_exists(doc_output_parent_folder)
doc_printer = DocumentationFilePrinter(doc_output_parent_folder=doc_output_parent_folder, doc_name='InstantaneousSpikeRateGroupsComputation')
doc_printer.save_documentation('InstantaneousSpikeRateGroupsComputation', across_session_inst_fr_computation, non_expanded_item_keys=['_reverse_cellID_index_map'])

In [ ]:
doc_printer = DocumentationFilePrinter(doc_output_parent_folder=doc_output_parent_folder, doc_name='SingleBarResult')
doc_printer.save_documentation('SingleBarResult', across_session_inst_fr_computation.Fig2_Replay_FR[0], non_expanded_item_keys=['_reverse_cellID_index_map'])
doc_printer.save_documentation('SingleBarResult', across_session_inst_fr_computation.Fig2_Replay_FR[1], non_expanded_item_keys=['_reverse_cellID_index_map'])
doc_printer.save_documentation('SingleBarResult', across_session_inst_fr_computation.Fig2_Replay_FR[2], non_expanded_item_keys=['_reverse_cellID_index_map'])
doc_printer.save_documentation('SingleBarResult', across_session_inst_fr_computation.Fig2_Replay_FR[3], non_expanded_item_keys=['_reverse_cellID_index_map'])

In [ ]:
across_session_inst_fr_computation.get_summary_dataframe()

In [ ]:
across_sessions_instantaneous_frs_list


In [ ]:
# print_keys_if_possible('InstantaneousSpikeRateGroupsComputation', across_session_inst_fr_computation, max_depth=3)



### 2024-09-10 - TODO: Filter out the datapoints from the bar plot corresponding to the novel vs. non-novel sessions

In [ ]:
import re
from typing import List, Tuple
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import ConciseSessionIdentifiers

# `across_session_inst_fr_computation.Fig2_Replay_FR`

novel_Fig2_Replay_FR = deepcopy(across_session_inst_fr_computation.Fig2_Replay_FR)
non_novel_Fig2_Replay_FR = deepcopy(across_session_inst_fr_computation.Fig2_Replay_FR)

# [0] and [1] both only use LxC_aclus
LxC_aclus = deepcopy(across_session_inst_fr_computation.Fig2_Replay_FR[0].LxC_aclus.tolist()) # ['a0s0_109', 'a0s1_3', 'a0s1_29', 'a0s1_103', 'a0s3_90', 'a0s4_91', 'a0s4_95', 'a1s1_23', 'a1s2_25', 'a1s3_14', 'a1s3_30', 'a1s3_32', 'a2s0_8', 'a2s0_27', 'a2s1_27']
_LxC_aclus_alt = deepcopy(across_session_inst_fr_computation.Fig2_Replay_FR[1].LxC_aclus.tolist()) # ['a0s0_109', 'a0s1_3', 'a0s1_29', 'a0s1_103', 'a0s3_90', 'a0s4_91', 'a0s4_95', 'a1s1_23', 'a1s2_25', 'a1s3_14', 'a1s3_30', 'a1s3_32', 'a2s0_8', 'a2s0_27', 'a2s1_27']
assert _LxC_aclus_alt == LxC_aclus

# [2] and [3] both only use SxC_aclus
SxC_aclus = deepcopy(across_session_inst_fr_computation.Fig2_Replay_FR[2].SxC_aclus.tolist()) # ['a0s0_109', 'a0s1_3', 'a0s1_29', 'a0s1_103', 'a0s3_90', 'a0s4_91', 'a0s4_95', 'a1s1_23', 'a1s2_25', 'a1s3_14', 'a1s3_30', 'a1s3_32', 'a2s0_8', 'a2s0_27', 'a2s1_27']
_SxC_aclus_alt = deepcopy(across_session_inst_fr_computation.Fig2_Replay_FR[3].SxC_aclus.tolist()) # ['a0s0_109', 'a0s1_3', 'a0s1_29', 'a0s1_103', 'a0s3_90', 'a0s4_91', 'a0s4_95', 'a1s1_23', 'a1s2_25', 'a1s3_14', 'a1s3_30', 'a1s3_32', 'a2s0_8', 'a2s0_27', 'a2s1_27']
assert _SxC_aclus_alt == SxC_aclus

parsed_LxC_aclus_df: pd.DataFrame = ConciseSessionIdentifiers.parse_concise_abbreviated_neuron_identifying_strings(LxC_aclus)
# parsed_LxC_aclus_df

parsed_SxC_aclus_df: pd.DataFrame = ConciseSessionIdentifiers.parse_concise_abbreviated_neuron_identifying_strings(SxC_aclus)
# parsed_SxC_aclus_df

# novel_LxC_indicies, non_novel_LxC_indicies = 
# partition_df(parsed_LxC_aclus_df, partitionColumn='is_session_novel')
# partition_df(parsed_SxC_aclus_df, partitionColumn='is_session_novel')

novel_LxC_indicies = np.where(parsed_LxC_aclus_df['is_session_novel'])[0]
non_novel_LxC_indicies = np.where(np.logical_not(parsed_LxC_aclus_df['is_session_novel']))[0]

(novel_LxC_indicies, non_novel_LxC_indicies)

novel_SxC_indicies = np.where(parsed_SxC_aclus_df['is_session_novel'])[0]
non_novel_SxC_indicies = np.where(np.logical_not(parsed_SxC_aclus_df['is_session_novel']))[0]

(novel_SxC_indicies, non_novel_SxC_indicies)

## OUTPUTS: (novel_LxC_indicies, non_novel_LxC_indicies), (novel_SxC_indicies, non_novel_SxC_indicies)

In [ ]:
curr_idx: int = 0
novel_Fig2_Replay_FR[curr_idx].LxC_aclus = novel_Fig2_Replay_FR[curr_idx].LxC_aclus[novel_LxC_indicies]
novel_Fig2_Replay_FR[curr_idx].values = novel_Fig2_Replay_FR[curr_idx].values[novel_LxC_indicies]
curr_idx: int = 1
novel_Fig2_Replay_FR[curr_idx].LxC_aclus = novel_Fig2_Replay_FR[curr_idx].LxC_aclus[novel_LxC_indicies]
novel_Fig2_Replay_FR[curr_idx].values = novel_Fig2_Replay_FR[curr_idx].values[novel_LxC_indicies]
curr_idx: int = 2
novel_Fig2_Replay_FR[curr_idx].SxC_aclus = novel_Fig2_Replay_FR[curr_idx].SxC_aclus[novel_SxC_indicies]
novel_Fig2_Replay_FR[curr_idx].values = novel_Fig2_Replay_FR[curr_idx].values[novel_SxC_indicies]
curr_idx: int = 3
novel_Fig2_Replay_FR[curr_idx].SxC_aclus = novel_Fig2_Replay_FR[curr_idx].SxC_aclus[novel_SxC_indicies]
novel_Fig2_Replay_FR[curr_idx].values = novel_Fig2_Replay_FR[curr_idx].values[novel_SxC_indicies]

across_session_inst_fr_computation.Fig2_Replay_FR = novel_Fig2_Replay_FR

In [ ]:
global_multi_session_context, _out_aggregate_fig_2 = AcrossSessionsVisualizations.across_sessions_bar_graphs(across_session_inst_fr_computation, num_sessions=num_sessions, enable_tiny_point_labels=True, enable_hover_labels=False, save_figure=False)

In [ ]:
plt.close('all')

In [ ]:
## group by 'is_novel_exposure'
## for `long_short_fr_indicies_analysis_table`
is_novel_partitioned_dfs = dict(zip(*partition_df(long_short_fr_indicies_analysis_table, partitionColumn='is_novel_exposure')))
novel_session_uids = is_novel_partitioned_dfs[True]['session_uid'].unique().tolist()
non_novel_session_uids = is_novel_partitioned_dfs[False]['session_uid'].unique().tolist()
## build dicts:
novel_only_inst_fr_dict = {k:v for k, v in across_sessions_instantaneous_fr_dict.items() if k.get_description(separator='|') in novel_session_uids}
non_novel_inst_fr_dict = {k:v for k, v in across_sessions_instantaneous_fr_dict.items() if k.get_description(separator='|') not in novel_session_uids}
# novel_only_inst_fr_dict

In [ ]:
# all
# graphics_output_dict |= AcrossSessionsVisualizations.across_sessions_firing_rate_index_figure(long_short_fr_indicies_analysis_results=long_short_fr_indicies_analysis_table, num_sessions=num_sessions, save_figure=False)
# split based on session novelty:
graphics_output_dict |= AcrossSessionsVisualizations.across_sessions_firing_rate_index_figure(long_short_fr_indicies_analysis_results=is_novel_partitioned_dfs[True], num_sessions=len(novel_session_uids), save_figure=False)
graphics_output_dict |= AcrossSessionsVisualizations.across_sessions_firing_rate_index_figure(long_short_fr_indicies_analysis_results=is_novel_partitioned_dfs[False], num_sessions=len(non_novel_session_uids), save_figure=False)


In [ ]:
graphics_output_dict |= AcrossSessionsVisualizations.across_sessions_long_and_short_firing_rate_replays_v_laps_figure(neuron_replay_stats_table=neuron_replay_stats_table, num_sessions=num_sessions, save_figure=True)


In [ ]:
## Aggregate across all of the sessions to build a new combined `InstantaneousSpikeRateGroupsComputation`, which can be used to plot the "PaperFigureTwo", bar plots for many sessions.
global_multi_session_context = IdentifyingContext(format_name='kdiba', num_sessions=num_sessions) # some global context across all of the sessions, not sure what to put here.



In [ ]:
_restore_previous_matplotlib_settings_callback = matplotlib_configuration_update(is_interactive=True, backend='Qt5Agg')
# _restore_previous_matplotlib_settings_callback = matplotlib_configuration_update(is_interactive=False, backend='AGG')

plt.show()


In [ ]:
plt.close('all')    

In [ ]:

matplotlib_configuration_update(is_interactive=True, backend='Qt5Agg')
matplotlib_output_container: MatplotlibRenderPlots = AcrossSessionsVisualizations.across_sessions_firing_rate_index_figure(long_short_fr_indicies_analysis_results=long_short_fr_indicies_analysis_table, num_sessions=num_sessions, save_figure=False)

In [ ]:
from scipy.stats import pearsonr

long_short_fr_indicies_analysis_table


In [ ]:
duplicates = long_short_fr_indicies_analysis_table[['x_frs_index', 'y_frs_index']].duplicated().sum()
print(f"Number of duplicate rows: {duplicates}")


In [ ]:
print(long_short_fr_indicies_analysis_table[['x_frs_index', 'y_frs_index']].std())


In [ ]:

# long_short_fr_indicies_analysis_table[['x_frs_index']].corrwith(long_short_fr_indicies_analysis_table['y_frs_index']) # long_short_fr_indicies_analysis_table[['x_frs_index']].corrwith(long_short_fr_indicies_analysis_table['y_frs_index'])
# Compute correlation and p-value
df_cleaned: pd.DataFrame = long_short_fr_indicies_analysis_table[['x_frs_index', 'y_frs_index']].dropna()
df_cleaned


In [ ]:
print(df_cleaned.shape)
corr, p_value = pearsonr(df_cleaned['x_frs_index'], df_cleaned['y_frs_index'])
print(f"Correlation: {corr}, P-value: {p_value}")
# Correlation: 0.48548405144431905, P-value: 1.0243398539233502e-48



In [ ]:
plt.close('all')

In [ ]:
graphics_output_dict.saved_figures[0][0] #.plot_data['saved_figures']

In [ ]:
from pyphocorehelpers.plotting.media_output_helpers import fig_to_clipboard

fig_to_clipboard(fig)


In [ ]:
# copy_image_to_clipboard(graphics_output_dict['figures'][0])
fig_to_clipboard(matplotlib_output_container.figures[0])

In [ ]:
matplotlib_configuration_update(is_interactive=True, backend='Qt5Agg')
graphics_output_dict = AcrossSessionsVisualizations.across_sessions_long_and_short_firing_rate_replays_v_laps_figure(neuron_replay_stats_table=neuron_replay_stats_table, num_sessions=num_sessions)

In [ ]:
_save_matplotlib_fig(graphics_output_dict)


In [ ]:
## Load the saved across-session results:
# Outputs: across_session_inst_fr_computation, across_sessions_instantaneous_fr_dict, across_sessions_instantaneous_frs_list, neuron_identities_table, long_short_fr_indicies_analysis_table, neuron_replay_stats_table

BATCH_DATE_TO_USE = f'2024-09-03'
inst_fr_output_filename: str = f'across_session_result_long_short_recomputed_inst_firing_rate_{BATCH_DATE_TO_USE}.pkl'

inst_fr_output_file = collected_outputs_directory.joinpath(inst_fr_output_filename).resolve()
Assert.path_exists(inst_fr_output_file)

across_session_inst_fr_computation, across_sessions_instantaneous_fr_dict, across_sessions_instantaneous_frs_list = AcrossSessionsResults.load_across_sessions_data(global_data_root_parent_path=collected_outputs_directory, inst_fr_output_filename=inst_fr_output_filename)
# across_sessions_instantaneous_fr_dict = loadData(global_batch_result_inst_fr_file_path)
num_sessions = len(across_sessions_instantaneous_fr_dict)
print(f'num_sessions: {num_sessions}')
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import 

## Load all across-session tables from the pickles:
output_path_suffix: str = f'{BATCH_DATE_TO_USE}'
neuron_identities_table, long_short_fr_indicies_analysis_table, neuron_replay_stats_table = AcrossSessionTables.load_all_combined_tables(override_output_parent_path=collected_outputs_directory, output_path_suffix=output_path_suffix) # output_path_suffix=f'2023-10-04-GL-Recomp'
num_sessions = len(neuron_replay_stats_table.session_uid.unique().to_numpy())
print(f'num_sessions: {num_sessions}')
# neuron_replay_stats_table


In [ ]:
neptuner.stop()

## 2024-09-04 - Batch Output Files

In [ ]:
from neuropy.core.user_annotations import UserAnnotationsManager
from pyphocorehelpers.Filesystem.metadata_helpers import FilesystemMetadata, get_file_metadata
from pyphocorehelpers.Filesystem.path_helpers import discover_data_files, generate_copydict, copy_movedict, copy_file, save_copydict_to_text_file, read_copydict_from_text_file, invert_filedict
from pyphoplacecellanalysis.General.Batch.runBatch import get_file_str_if_file_exists
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import check_output_h5_files, copy_files_in_filelist_to_dest
from pyphoplacecellanalysis.General.Batch.runBatch import ConcreteSessionFolder, BackupMethods

# a_batch_progress_df = included_session_batch_progress_df.copy()
included_session_contexts
h5_contexts_paths_dict
# h5_session_contexts

debug_print = False
known_global_data_root_parent_paths = [Path(r'/nfs/turbo/umms-kdiba/Data'), Path(r'W:\Data'), Path(r'/home/halechr/cloud/turbo/Data'), Path(r'/media/halechr/MAX/Data'), Path(r'/Volumes/MoverNew/data')] # , Path(r'/home/halechr/FastData'), Path(r'/home/halechr/turbo/Data'), Path(r'W:\Data'), Path(r'/home/halechr/cloud/turbo/Data')
global_data_root_parent_path = find_first_extant_path(known_global_data_root_parent_paths)
assert global_data_root_parent_path.exists(), f"global_data_root_parent_path: {global_data_root_parent_path} does not exist! Is the right computer's config commented out above?"
## Build Pickle Path:
# Hardcoded included_session_contexts:
included_session_contexts = UserAnnotationsManager.get_hardcoded_good_sessions()
good_session_concrete_folders = ConcreteSessionFolder.build_concrete_session_folders(global_data_root_parent_path, included_session_contexts)

# Output Paths:
included_h5_paths = [get_file_str_if_file_exists(v.pipeline_results_h5) for v in good_session_concrete_folders]
# copy_dict = ConcreteSessionFolder.build_backup_copydict(good_session_concrete_folders, backup_mode=BackupMethods.RenameInSourceDirectory, only_include_file_types=['local_pkl', 'global_pkl'])
check_output_h5_files(included_h5_paths)

## OUTPUTS: included_h5_paths, included_session_contexts, good_session_concrete_folders

included_h5_paths


RESULT_DATE_TO_USE = '2024-09-04'
# RESULT_DATE_TO_USE = '2024-07-02'

# included_file_types_paths_dict = {'h5': included_h5_paths, 'recomputed_inst_fr_comps': []}
# included_file_types_paths_dict['recomputed_inst_fr_comps'] = [get_file_path_if_file_exists(v.output_folder.joinpath(f'{RESULT_DATE_TO_USE}_recomputed_inst_fr_comps_0.0005.h5').resolve()) for v in good_session_concrete_folders]
# included_file_types_paths_dict


In [ ]:
## INPUTS: good_session_concrete_folders, target_dir, BATCH_DATE_TO_USE, custom_file_types_dict
from pyphoplacecellanalysis.General.Batch.runBatch import get_file_path_if_file_exists
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import copy_session_folder_files_to_target_dir

custom_file_types_dict = {'recomputed_inst_fr_comps': (lambda a_session_folder: get_file_path_if_file_exists(a_session_folder.output_folder.joinpath(f'{RESULT_DATE_TO_USE}_recomputed_inst_fr_comps_0.0005.pkl').resolve())),
						#   'PHONEW.evt': (lambda a_session_folder: get_file_path_if_file_exists(a_session_folder.output_folder.joinpath(f'{a_session_folder.context.session_name}.PHONEW.evt').resolve())),
						  }

# target_dir: Path = Path(global_data_root_parent_path)
target_dir: Path = collected_outputs_directory
moved_files_dict_files, (filelist_path, filedict_out_path) = copy_session_folder_files_to_target_dir(good_session_concrete_folders, target_dir=target_dir, RESULT_DATE_TO_USE=BATCH_DATE_TO_USE, custom_file_types_dict=custom_file_types_dict, dry_run=False)

## Extract `across_sessions_instantaneous_fr_dict` from the computation outputs

# Plotting via Plotly
`!pip install kaleido=="v0.1.0.post1" `

In [43]:
import plotly.io as pio
from pyphoplacecellanalysis.Pho2D.plotly.plotly_templates import PlotlyHelpers
from pyphoplacecellanalysis.Pho2D.statistics_plotting_helpers import plot_histograms_across_sessions, plot_stacked_histograms
from pyphoplacecellanalysis.Pho2D.plotly.Extensions.plotly_helpers import plotly_helper_save_figures, _helper_build_figure, plotly_pre_post_delta_scatter, plot_across_sessions_scatter_results

# fig_size_kwargs = {'width': 1650, 'height': 480}
resolution_multiplier = 1
# fig_size_kwargs = {'width': resolution_multiplier*1650, 'height': resolution_multiplier*480}
fig_size_kwargs = {'width': resolution_multiplier*1920, 'height': resolution_multiplier*480}
is_dark_mode, template = PlotlyHelpers.get_plotly_template(is_dark_mode=False)
pio.templates.default = template

# figure_export_path = Path(r'E:\Dropbox (Personal)\Active\Kamran Diba Lab\Presentations\2024-05-30 - Pho iNAV Poster\Figures').resolve()
# figure_export_path = Path('/Users/pho/Dropbox (Personal)/Active/Kamran Diba Lab/Presentations/2024-05-30 - Pho iNAV Poster/Figures').resolve()

# assert figure_export_path.exists()

from PIL import Image

from pyphocorehelpers.programming_helpers import copy_image_to_clipboard

def save_plotly(a_fig, a_fig_context):
    """ 
    captures: TODAY_DAY_DATE, figures_folder, neptuner_run
    """
    fig_save_path: Path = figures_folder.joinpath('_'.join([TODAY_DAY_DATE, sanitize_filename_for_Windows(a_fig_context.get_description())])).resolve()
    figure_out_paths = {'.html': fig_save_path.with_suffix('.html'), '.png': fig_save_path.with_suffix('.png')}
    a_fig.write_html(figure_out_paths['.html'])
    display(fullwidth_path_widget(figure_out_paths['.html'], file_name_label='.html'))
    # print(file_uri_from_path(figure_out_paths['.html']))
    a_fig.write_image(figure_out_paths['.png'])
    # print(file_uri_from_path(figure_out_paths['.png']))
    display(fullwidth_path_widget(figure_out_paths['.png'], file_name_label='.png'))

    if neptuner_run is not None:
        a_full_figure_path_key: str = a_fig_context.get_description(separator='/', include_property_names=True, key_value_separator=':') # .replace(' ', '_')
        # a_full_figure_path_key: str = a_fig_context.get_description(separator=':', include_property_names=True, key_value_separator='|')
        print(f'a_full_figure_path_key: "{a_full_figure_path_key}"')
        # neptuner_run['outputs']['figures'][f"{a_full_figure_path_key}"].upload(a_fig)
        neptuner_run['outputs']['figures'][f"{a_full_figure_path_key}"].upload(figure_out_paths['.html'].as_posix())
        # neptuner.figures[f"{a_full_figure_path_key}"].upload(a_fig)
        
    return figure_out_paths


In [30]:
# 'short_best_wcorr', 'long_best_wcorr', 'wcorr_diff'
# 'pearsonr_long_LR', 'long_best_wcorr', 'wcorr_diff'
all_sessions_all_scores_ripple_df

,start,stop,label,duration,is_valid_epoch,P_LR,P_RL,P_Long,P_Short,P_Long_LR,congruent_dir_bins_ratio_long_LR,coverage_long_LR,direction_change_bin_ratio_long_LR,integral_second_derivative_long_LR,jump_long_LR,longest_sequence_length_ratio_long_LR,pearsonr_long_LR,stddev_of_diff_long_LR,total_congruent_direction_change_long_LR,total_variation_long_LR,travel_long_LR,wcorr_long_LR,P_Long_RL,congruent_dir_bins_ratio_long_RL,coverage_long_RL,direction_change_bin_ratio_long_RL,integral_second_derivative_long_RL,jump_long_RL,longest_sequence_length_ratio_long_RL,pearsonr_long_RL,stddev_of_diff_long_RL,total_congruent_direction_change_long_RL,total_variation_long_RL,travel_long_RL,wcorr_long_RL,P_Short_LR,congruent_dir_bins_ratio_short_LR,coverage_short_LR,direction_change_bin_ratio_short_LR,integral_second_derivative_short_LR,jump_short_LR,longest_sequence_length_ratio_short_LR,pearsonr_short_LR,stddev_of_diff_short_LR,total_congruent_direction_change_short_LR,total_variation_short_LR,travel_short_LR,wcorr_short_LR,P_Short_RL,congruent_dir_bins_ratio_short_RL,...,score_diff,long_best_velocity,short_best_velocity,velocity_diff,long_best_intercept,short_best_intercept,intercept_diff,long_best_speed,short_best_speed,speed_diff,long_best_wcorr,short_best_wcorr,wcorr_diff,long_best_pearsonr,short_best_pearsonr,pearsonr_diff,long_best_travel,short_best_travel,travel_diff,long_best_coverage,short_best_coverage,coverage_diff,long_best_jump,short_best_jump,jump_diff,long_best_longest_sequence_length_ratio,short_best_longest_sequence_length_ratio,longest_sequence_length_ratio_diff,long_best_direction_change_bin_ratio,short_best_direction_change_bin_ratio,direction_change_bin_ratio_diff,long_best_congruent_dir_bins_ratio,short_best_congruent_dir_bins_ratio,congruent_dir_bins_ratio_diff,long_best_total_congruent_direction_change,short_best_total_congruent_direction_change,total_congruent_direction_change_diff,long_best_total_variation,short_best_total_variation,total_variation_diff,long_best_integral_second_derivative,short_best_integral_second_derivative,integral_second_derivative_diff,long_best_stddev_of_diff,short_best_stddev_of_diff,stddev_of_diff_diff,session_experience_rank,session_experience_orientation_rank,is_novel_exposure,pre_post_delta_category
4927,40.187177,40.488202,0,0.301025,True,0.617735,0.382265,0.544716,0.455284,0.336490,0.363636,0.157895,0.090909,241946.584175,0.004673,0.500000,0.129413,86.039397,0.000000,439.990677,0.186912,0.214083,0.208226,0.454545,0.350877,0.454545,540635.591373,0.004673,0.250000,-0.360773,142.173880,610.676716,1198.595294,0.509174,-0.348438,0.281245,0.545455,0.210526,0.181818,84523.766547,0.004340,0.500000,0.147619,52.041595,163.099992,303.441846,0.191567,0.124031,0.174039,0.454545,...,-0.033242,13.792811203715296,-13.792811203715296,0.0,786.8846318537384,-344.80661350696505,442.0780183467733,13.792811203715296,13.792811203715296,0.0,-0.348438,-0.201241,0.147198,-0.360773,-0.272826,0.087947,0.509174,0.617803,-0.108629,0.350877,0.333333,0.017544,0.004673,0.005580,-0.000907,0.250000,0.250000,0.000000,0.454545,0.454545,0.000000,0.454545,0.454545,0.000000,610.676716,496.886024,1.137907e+02,1198.595294,978.599955,2.199953e+02,540635.591373,353100.732319,187534.859054,142.173880,113.517643,28.656237,2.0,1.0,False,pre-delta
4928,41.011916,41.359114,1,0.347197,True,0.471784,0.528216,0.528794,0.471206,0.249477,0.333333,0.228070,0.416667,472095.808593,0.004673,0.307692,-0.041734,128.065639,603.090670,1198.595294,0.466743,0.028805,0.279317,0.500000,0.315789,0.500000,311910.682340,0.004172,0.153846,-0.184740,108.828847,606.883693,1001.358093,0.389937,-0.416560,0.222308,0.333333,0.228070,0.416667,225804.343142,0.005208,0.384615,-0.075975,89.148931,352.751147,701.709270,0.406082,-0.052581,0.248898,0.583333,...,0.029490,88.50387189050716,25.286820540144912,63.21705135036224,3878.4098537356113,1254.6230383766717,2623.7868153589397,88.50387189050716,25.286820540144912,63.21705135036224,0.028805,-0.052581,-0.023776,-0.0

In [44]:
concatenated_ripple_df = deepcopy(filtered_all_sessions_all_scores_ripple_df)
concatenated_ripple_df['session_name'].unique()

# ['kdiba_gor01_one_2006-6-08_14-26-15',
# 'kdiba_gor01_one_2006-6-08_14-26-15_None',
# 'kdiba_gor01_one_2006-6-09_1-22-43',
# 'kdiba_gor01_one_2006-6-12_15-55-31',
# 'kdiba_gor01_two_2006-6-07_16-40-19',
# 'kdiba_gor01_two_2006-6-08_21-16-25',
# 'kdiba_gor01_two_2006-6-09_22-24-40',
# 'kdiba_pin01_one_11-02_19-28-0',
# 'kdiba_pin01_one_fet11-01_12-58-54']

array(['kdiba_gor01_one_2006-6-07_11-26-53',
       'kdiba_gor01_one_2006-6-07_11-26-53_None',
       'kdiba_gor01_one_2006-6-07_11-26-53_long_short_firing_rate_indicies',
       'kdiba_gor01_one_2006-6-08_14-26-15',
       'kdiba_gor01_one_2006-6-08_14-26-15_None',
       'kdiba_gor01_one_2006-6-09_1-22-43',
       'kdiba_gor01_one_2006-6-09_1-22-43_None',
       'kdiba_gor01_one_2006-6-12_15-55-31',
       'kdiba_gor01_two_2006-6-07_16-40-19',
       'kdiba_gor01_two_2006-6-08_21-16-25',
       'kdiba_gor01_two_2006-6-09_22-24-40',
       'kdiba_gor01_two_2006-6-12_16-53-46',
       'kdiba_pin01_one_11-02_19-28-0',
       'kdiba_pin01_one_fet11-01_12-58-54',
       'kdiba_vvp01_one_2006-4-09_17-29-30',
       'kdiba_vvp01_one_2006-4-10_12-25-50',
       'kdiba_vvp01_two_2006-4-09_16-40-54',
       'kdiba_vvp01_two_2006-4-10_12-58-3',
       'kdiba_vvp01_two_2006-4-16_18-47-52',
       'kdiba_vvp01_two_2006-4-28_12-38-13',
       'kdiba_vvp01_two_2006-4-28_12-38-13_None'], dtype=objec

In [46]:
histogram_bins = 25
num_sessions = 1

# concatenated_ripple_df = deepcopy(df)
## INPUTS: all_sessions_all_scores_ripple_df

# concatenated_ripple_df = deepcopy(concatenated_ripple_df)
# concatenated_ripple_df = deepcopy(all_sessions_ripple_df)

# concatenated_ripple_df = deepcopy(filtered_all_sessions_wcorr_ripple_df)

concatenated_ripple_df = deepcopy(filtered_all_sessions_all_scores_ripple_df)


# concatenated_ripple_df = deepcopy(all_sessions_all_scores_ripple_df)
# concatenated_ripple_df = concatenated_ripple_df[concatenated_ripple_df['custom_replay_name'] != ''] # non-custom
# concatenated_ripple_df = deepcopy(filtered_all_sessions_all_scores_ripple_df)

# concatenated_ripple_df = deepcopy(all_sessions_ripple_time_bin_df)
# concatenated_ripple_df = deepcopy(all_sessions_all_scores_ripple_df)
# concatenated_ripple_df = deepcopy(single_time_bin_size_all_sessions_ripple_df)
display(concatenated_ripple_df)
# ['longest_sequence_length_ratio_diff', 'direction_change_bin_ratio_diff', 'congruent_dir_bins_ratio_diff', 'total_congruent_direction_change_diff']


# variable_name = 'P_Long'
variable_name = 'P_Short' # Shows expected effect - short-only replay prior to delta and then split replays post-delta
# variable_name = 'short_best_wcorr'
# variable_name = 'long_best_pf_peak_x_pearsonr'
# variable_name = 'long_best_jump'
# variable_name = 'wcorr_abs_diff'
# variable_name = 'pearsonr_abs_diff'
# variable_name = 'direction_change_bin_ratio_diff'
# variable_name = 'longest_sequence_length_ratio_diff'
# variable_name = 'long_best_longest_sequence_length_ratio'
# variable_name = 'long_best_congruent_dir_bins_ratio'
# variable_name = 'congruent_dir_bins_ratio_diff'
# variable_name = 'total_congruent_direction_change_diff'
# variable_name = 'long_best_congruent_dir_bins_ratio'
# variable_name = 'long_best_direction_change_bin_ratio'
# variable_name = 'long_best_congruent_dir_bins_ratio'
# 'color':'is_user_annotated_epoch'
# 'color': 'is_user_annotated_epoch', 


# concatenated_ripple_df = concatenated_ripple_df[concatenated_ripple_df[variable_name].abs() > 0.25]

y_baseline_level: float = 0.5 # for P(short), etc
# y_baseline_level: float = 0.0 # for wcorr, etc

# px_scatter_kwargs = {'x': 'delta_aligned_start_t', 'y': variable_name, 'color':"is_user_annotated_epoch", 'title': f"'{variable_name}'"} # , 'color': 'time_bin_size', 'range_y': [-1.0, 1.0], 'labels': {'session_name': 'Session', 'time_bin_size': 'tbin_size', 'is_user_annotated_epoch':'user_sel'}
px_scatter_kwargs = {'x': 'delta_aligned_start_t', 'y': variable_name, 'title': f"'{variable_name}'"} # , 'color': 'time_bin_size', 'range_y': [-1.0, 1.0], 'labels': {'session_name': 'Session', 'time_bin_size': 'tbin_size', 'is_user_annotated_epoch':'user_sel'}
# px_scatter_kwargs.pop('color')


# hist_kwargs = dict(color="time_bin_size")
hist_kwargs = dict(color="is_user_annotated_epoch") # , histnorm='probability density'
hist_kwargs.pop('color')

# px_scatter_kwargs['color'] = 'custom_replay_name'
# hist_kwargs['color'] = 'custom_replay_name'


# px_scatter_kwargs['color'] = 'session_name'
# hist_kwargs['color'] = 'session_name'

px_scatter_kwargs['color'] = 'session_experience_rank'
hist_kwargs['color'] = 'session_experience_rank'

# px_scatter_kwargs['color'] = 'session_experience_orientation_rank'
# hist_kwargs['color'] = 'session_experience_orientation_rank'


new_fig_ripples, new_fig_ripples_context = plotly_pre_post_delta_scatter(data_results_df=concatenated_ripple_df, out_scatter_fig=None, histogram_bins=histogram_bins,
                        px_scatter_kwargs=px_scatter_kwargs, histogram_variable_name=variable_name, hist_kwargs=hist_kwargs, forced_range_y=None,
                        time_delta_tuple=(earliest_delta_aligned_t_start, 0.0, latest_delta_aligned_t_end), legend_title_text=None, is_dark_mode=is_dark_mode)

new_fig_ripples = new_fig_ripples.update_layout(fig_size_kwargs)
_extras_output_dict = {}
if is_dark_mode:
    _extras_output_dict["y_mid_line"] = new_fig_ripples.add_hline(y=y_baseline_level, line=dict(color="rgba(0.8,0.8,0.8,.75)", width=2), row='all', col='all')
else:
    _extras_output_dict["y_mid_line"] = new_fig_ripples.add_hline(y=y_baseline_level, line=dict(color="rgba(0.2,0.2,0.2,.75)", width=2), row='all', col='all')

# # Update layout to add a title to the legend
# new_fig_ripples.update_layout(
#     legend_title_text='Is User Selected'  # Add a title to the legend
# )

# fig_to_clipboard(new_fig_ripples, **fig_size_kwargs)
new_fig_ripples_context = new_fig_ripples_context.adding_context_if_missing(num_sessions=num_sessions, plot_type='scatter+hist', comparison='pre-post-delta', variable_name=variable_name)
figure_out_paths = save_plotly(a_fig=new_fig_ripples, a_fig_context=new_fig_ripples_context)
new_fig_ripples

,start,stop,label,duration,is_valid_epoch,P_LR,P_RL,P_Long,P_Short,P_Long_LR,congruent_dir_bins_ratio_long_LR,coverage_long_LR,direction_change_bin_ratio_long_LR,integral_second_derivative_long_LR,jump_long_LR,longest_sequence_length_ratio_long_LR,pearsonr_long_LR,stddev_of_diff_long_LR,total_congruent_direction_change_long_LR,total_variation_long_LR,travel_long_LR,wcorr_long_LR,P_Long_RL,congruent_dir_bins_ratio_long_RL,coverage_long_RL,direction_change_bin_ratio_long_RL,integral_second_derivative_long_RL,jump_long_RL,longest_sequence_length_ratio_long_RL,pearsonr_long_RL,stddev_of_diff_long_RL,total_congruent_direction_change_long_RL,total_variation_long_RL,travel_long_RL,wcorr_long_RL,P_Short_LR,congruent_dir_bins_ratio_short_LR,coverage_short_LR,direction_change_bin_ratio_short_LR,integral_second_derivative_short_LR,jump_short_LR,longest_sequence_length_ratio_short_LR,pearsonr_short_LR,stddev_of_diff_short_LR,total_congruent_direction_change_short_LR,total_variation_short_LR,travel_short_LR,wcorr_short_LR,P_Short_RL,congruent_dir_bins_ratio_short_RL,...,score_diff,long_best_velocity,short_best_velocity,velocity_diff,long_best_intercept,short_best_intercept,intercept_diff,long_best_speed,short_best_speed,speed_diff,long_best_wcorr,short_best_wcorr,wcorr_diff,long_best_pearsonr,short_best_pearsonr,pearsonr_diff,long_best_travel,short_best_travel,travel_diff,long_best_coverage,short_best_coverage,coverage_diff,long_best_jump,short_best_jump,jump_diff,long_best_longest_sequence_length_ratio,short_best_longest_sequence_length_ratio,longest_sequence_length_ratio_diff,long_best_direction_change_bin_ratio,short_best_direction_change_bin_ratio,direction_change_bin_ratio_diff,long_best_congruent_dir_bins_ratio,short_best_congruent_dir_bins_ratio,congruent_dir_bins_ratio_diff,long_best_total_congruent_direction_change,short_best_total_congruent_direction_change,total_congruent_direction_change_diff,long_best_total_variation,short_best_total_variation,total_variation_diff,long_best_integral_second_derivative,short_best_integral_second_derivative,integral_second_derivative_diff,long_best_stddev_of_diff,short_best_stddev_of_diff,stddev_of_diff_diff,session_experience_rank,session_experience_orientation_rank,is_novel_exposure,pre_post_delta_category
0,19.584635,19.710587,1,0.125952,True,0.509376,0.490624,0.861733,0.138267,0.438946,0.500000,0.245614,0.500000,117853.583616,0.003087,0.571429,NaN,94.395864,0.000000,453.221023,0.352976,0.115717,0.422787,0.666667,0.263158,0.166667,71365.465847,0.003254,0.714286,NaN,73.656486,0.000000,294.593665,0.229434,1.476788e-01,0.070430,0.333333,0.280702,0.500000,113246.138989,0.005952,0.571429,NaN,103.563241,226.610512,423.006289,0.489591,0.106943,0.067837,0.666667,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,True,pre-delta
1,28.343394,28.453003,2,0.109609,True,0.554285,0.445715,0.880086,0.119914,0.487819,0.400000,0.263158,0.400000,73505.146015,0.003171,0.666667,-0.038696,67.671901,200.172619,256.825247,0.240024,-0.648294,0.392267,0.400000,0.315789,0.600000,173314.093553,0.003254,0.500000,0.582680,102.602273,0.000000,430.559972,0.402392,-2.344744e-02,0.066467,0.600000,0.210526,0.400000,183627.351960,0.005828,0.666667,-0.167127,105.848646,305.924191,430.559972,0.598000,-0.620315,0.053447,0.400000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,True,pre-delta
2,57.231484,57.443913,3,0.212429,False,0.379173,0.620827,0.479174,0.520826,0.181690,0.166667,0.614035,0.166667,88240.410100,0.003421,0.846154,-0.573668,62.298018,196.395777,351.246293,0.136778,0.056294,0.297484,1.083333,0.526316,0.000000,0.000000,0.000000,1.000000,-0.015647,0.000000,0.000000,0.000000,0.000000,1.122023e-18,0.197484,0.166667,0.754386,0.166667,

'is_user_annotated_epoch'

num_unique_sessions: 21, num_unique_time_bins: 1
num_unique_sessions: 21, num_unique_time_bins: 1
len(pre_delta_fig.data): 11


Box(children=(Label(value='.html', layout=Layout(width='auto')), HTML(value="<b style='font-size: smaller;'>K:…

Box(children=(Label(value='.png', layout=Layout(width='auto')), HTML(value="<b style='font-size: smaller;'>K:\…

In [47]:
histogram_bins = 25
num_sessions = 1

# concatenated_ripple_df = deepcopy(filtered_all_sessions_wcorr_ripple_df)
concatenated_ripple_df = deepcopy(filtered_all_sessions_simple_pearson_ripple_df) # ['P_LR', 'P_RL', 'P_Long', 'P_Short', 'ripple_idx', 'ripple_start_t', 'P_Long_LR', 'P_Long_RL', 'P_Short_LR', 'P_Short_RL', 'most_likely_decoder_index', 'start', 'stop', 'label', 'duration', 'long_LR_pf_peak_x_pearsonr', 'long_RL_pf_peak_x_pearsonr', 'short_LR_pf_peak_x_pearsonr', 'short_RL_pf_peak_x_pearsonr', 'best_decoder_index', 'session_name', 'time_bin_size', 'delta_aligned_start_t', 'is_user_annotated_epoch', 'is_valid_epoch', 'custom_replay_name', 'epoch_idx', 'long_best_pf_peak_x_pearsonr', 'short_best_pf_peak_x_pearsonr', 'wcorr_long_LR', 'wcorr_long_RL', 'wcorr_short_LR', 'wcorr_short_RL', 'long_best_wcorr', 'short_best_wcorr', 'wcorr_abs_diff', 'pearsonr_abs_diff']

print(f'concatenated_ripple_df.columns: {list(concatenated_ripple_df.columns)}')
# variable_name = 'P_Long'
variable_name = 'P_Short' # Shows expected effect - short-only replay prior to delta and then split replays post-delta
# variable_name = 'short_best_wcorr'
# variable_name = 'long_best_pf_peak_x_pearsonr'
# variable_name = 'long_best_jump'
# variable_name = 'wcorr_abs_diff'
# variable_name = 'short_best_wcorr'
# variable_name = 'wcorr_long_LR'
# variable_name = 'pearsonr_abs_diff'
# variable_name = 'direction_change_bin_ratio_diff'
# variable_name = 'longest_sequence_length_ratio_diff'
# variable_name = 'long_best_longest_sequence_length_ratio'
# variable_name = 'long_best_congruent_dir_bins_ratio'
# variable_name = 'congruent_dir_bins_ratio_diff'
# variable_name = 'total_congruent_direction_change_diff'
# variable_name = 'long_best_congruent_dir_bins_ratio'
# variable_name = 'long_best_direction_change_bin_ratio'
# variable_name = 'long_best_congruent_dir_bins_ratio'
# 'color':'is_user_annotated_epoch'
# 'color': 'is_user_annotated_epoch', 

y_baseline_level: float = 0.5 # for P(short), etc
# y_baseline_level: float = 0.0 # for wcorr, etc

# px_scatter_kwargs = {'x': 'delta_aligned_start_t', 'y': variable_name, 'color':"is_user_annotated_epoch", 'title': f"'{variable_name}'"} # , 'color': 'time_bin_size', 'range_y': [-1.0, 1.0], 'labels': {'session_name': 'Session', 'time_bin_size': 'tbin_size', 'is_user_annotated_epoch':'user_sel'}
px_scatter_kwargs = {'x': 'delta_aligned_start_t', 'y': variable_name, 'title': f"'{variable_name}'"} # , 'color': 'time_bin_size', 'range_y': [-1.0, 1.0], 'labels': {'session_name': 'Session', 'time_bin_size': 'tbin_size', 'is_user_annotated_epoch':'user_sel'}
# px_scatter_kwargs.pop('color')

# hist_kwargs = dict(color="time_bin_size")
hist_kwargs = dict(color="is_user_annotated_epoch") # , histnorm='probability density'
hist_kwargs.pop('color')
new_fig_ripples, new_fig_ripples_context = plotly_pre_post_delta_scatter(data_results_df=concatenated_ripple_df, out_scatter_fig=None, histogram_bins=histogram_bins,
                        px_scatter_kwargs=px_scatter_kwargs, histogram_variable_name=variable_name, hist_kwargs=hist_kwargs, forced_range_y=None,
                        time_delta_tuple=(earliest_delta_aligned_t_start, 0.0, latest_delta_aligned_t_end), legend_title_text=None, is_dark_mode=is_dark_mode)

new_fig_ripples = new_fig_ripples.update_layout(fig_size_kwargs)

_extras_output_dict = {}
if is_dark_mode:
    _extras_output_dict["y_mid_line"] = new_fig_ripples.add_hline(y=y_baseline_level, line=dict(color="rgba(0.8,0.8,0.8,.75)", width=2), row='all', col='all')
else:
    _extras_output_dict["y_mid_line"] = new_fig_ripples.add_hline(y=y_baseline_level, line=dict(color="rgba(0.2,0.2,0.2,.75)", width=2), row='all', col='all')


# fig_to_clipboard(new_fig_ripples, **fig_size_kwargs)
new_fig_ripples_context = new_fig_ripples_context.adding_context_if_missing(num_sessions=num_sessions, plot_type='scatter+hist', comparison='pre-post-delta', variable_name=variable_name)
# figure_out_paths = save_plotly(a_fig=new_fig_ripples, a_fig_context=new_fig_ripples_context)
new_fig_ripples

concatenated_ripple_df.columns: ['P_LR', 'P_RL', 'P_Long', 'P_Short', 'ripple_idx', 'ripple_start_t', 'P_Long_LR', 'P_Long_RL', 'P_Short_LR', 'P_Short_RL', 'most_likely_decoder_index', 'start', 'stop', 'label', 'duration', 'end', 'long_LR_pf_peak_x_pearsonr', 'long_RL_pf_peak_x_pearsonr', 'short_LR_pf_peak_x_pearsonr', 'short_RL_pf_peak_x_pearsonr', 'best_decoder_index', 'session_name', 'time_bin_size', 'delta_aligned_start_t', 'custom_replay_name', 'is_valid_epoch', 'is_user_annotated_epoch', 'epoch_idx', 'long_best_pf_peak_x_pearsonr', 'short_best_pf_peak_x_pearsonr', 'wcorr_long_LR', 'wcorr_long_RL', 'wcorr_short_LR', 'wcorr_short_RL', 'long_best_wcorr', 'short_best_wcorr', 'wcorr_abs_diff', 'pearsonr_abs_diff', 'session_experience_rank', 'session_experience_orientation_rank', 'is_novel_exposure']


'is_user_annotated_epoch'

num_unique_sessions: 20, num_unique_time_bins: 1
num_unique_sessions: 20, num_unique_time_bins: 1
len(pre_delta_fig.data): 1


In [58]:
from pyphoplacecellanalysis.Pho2D.plotly.Extensions.plotly_helpers import plotly_pre_post_delta_scatter

histogram_bins = 25
num_sessions = 1

# ==================================================================================================================== #
# Laps                                                                                                                 #
# ==================================================================================================================== #
# all_sessions_laps_time_bin_df
# all_sessions_simple_pearson_laps_df

concatenated_ripple_df = deepcopy(all_sessions_simple_pearson_laps_df) # ['P_LR', 'P_RL', 'P_Long', 'P_Short', 'ripple_idx', 'ripple_start_t', 'P_Long_LR', 'P_Long_RL', 'P_Short_LR', 'P_Short_RL', 'most_likely_decoder_index', 'start', 'stop', 'label', 'duration', 'long_LR_pf_peak_x_pearsonr', 'long_RL_pf_peak_x_pearsonr', 'short_LR_pf_peak_x_pearsonr', 'short_RL_pf_peak_x_pearsonr', 'best_decoder_index', 'session_name', 'time_bin_size', 'delta_aligned_start_t', 'is_user_annotated_epoch', 'is_valid_epoch', 'custom_replay_name', 'epoch_idx', 'long_best_pf_peak_x_pearsonr', 'short_best_pf_peak_x_pearsonr', 'wcorr_long_LR', 'wcorr_long_RL', 'wcorr_short_LR', 'wcorr_short_RL', 'long_best_wcorr', 'short_best_wcorr', 'wcorr_abs_diff', 'pearsonr_abs_diff']
# concatenated_ripple_df = deepcopy(all_sessions_laps_time_bin_df) # ['P_LR', 'P_RL', 'P_Long', 'P_Short', 'ripple_idx', 'ripple_start_t', 'P_Long_LR', 'P_Long_RL', 'P_Short_LR', 'P_Short_RL', 'most_likely_decoder_index', 'start', 'stop', 'label', 'duration', 'long_LR_pf_peak_x_pearsonr', 'long_RL_pf_peak_x_pearsonr', 'short_LR_pf_peak_x_pearsonr', 'short_RL_pf_peak_x_pearsonr', 'best_decoder_index', 'session_name', 'time_bin_size', 'delta_aligned_start_t', 'is_user_annotated_epoch', 'is_valid_epoch', 'custom_replay_name', 'epoch_idx', 'long_best_pf_peak_x_pearsonr', 'short_best_pf_peak_x_pearsonr', 'wcorr_long_LR', 'wcorr_long_RL', 'wcorr_short_LR', 'wcorr_short_RL', 'long_best_wcorr', 'short_best_wcorr', 'wcorr_abs_diff', 'pearsonr_abs_diff']
print(f'concatenated_ripple_df.columns: {list(concatenated_ripple_df.columns)}')
concatenated_ripple_df

# variable_name = 'P_Long'
# variable_name = 'P_Short' # Shows expected effect - short-only replay prior to delta and then split replays post-delta
variable_name = 'P_LR'

y_baseline_level: float = 0.5 # for P(short), etc
# y_baseline_level: float = 0.0 # for wcorr, etc

# px_scatter_kwargs = {'x': 'delta_aligned_start_t', 'y': variable_name, 'color':"is_user_annotated_epoch", 'title': f"'{variable_name}'"} # , 'color': 'time_bin_size', 'range_y': [-1.0, 1.0], 'labels': {'session_name': 'Session', 'time_bin_size': 'tbin_size', 'is_user_annotated_epoch':'user_sel'}
px_scatter_kwargs = {'x': 'delta_aligned_start_t', 'y': variable_name, 'title': f"Lap Individual Time Bins - '{variable_name}'"} # , 'color': 'time_bin_size', 'range_y': [-1.0, 1.0], 'labels': {'session_name': 'Session', 'time_bin_size': 'tbin_size', 'is_user_annotated_epoch':'user_sel'}
# px_scatter_kwargs['color'] = "time_bin_size"
# px_scatter_kwargs.pop('color')

hist_kwargs = dict(color="time_bin_size")
# hist_kwargs = dict(color="is_user_annotated_epoch") # , histnorm='probability density'
hist_kwargs.pop('color')
new_fig_laps, new_fig_laps_context = plotly_pre_post_delta_scatter(data_results_df=concatenated_ripple_df, out_scatter_fig=None, histogram_bins=histogram_bins,
                        px_scatter_kwargs=px_scatter_kwargs, histogram_variable_name=variable_name, hist_kwargs=hist_kwargs, forced_range_y=None,
                        time_delta_tuple=(earliest_delta_aligned_t_start, 0.0, latest_delta_aligned_t_end), legend_title_text=None, is_dark_mode=is_dark_mode)

new_fig_laps = new_fig_laps.update_layout(fig_size_kwargs)

_extras_output_dict = {}
if is_dark_mode:
    _extras_output_dict["y_mid_line"] = new_fig_laps.add_hline(y=y_baseline_level, line=dict(color="rgba(0.8,0.8,0.8,.75)", width=2), row='all', col='all')
else:
    _extras_output_dict["y_mid_line"] = new_fig_laps.add_hline(y=y_baseline_level, line=dict(color="rgba(0.2,0.2,0.2,.75)", width=2), row='all', col='all')


fig_to_clipboard(new_fig_laps, **fig_size_kwargs)
new_fig_laps_context = new_fig_laps_context.adding_context_if_missing(num_sessions=num_sessions, plot_type='scatter+hist', comparison='pre-post-delta', variable_name=variable_name)
figure_out_paths = save_plotly(a_fig=new_fig_laps, a_fig_context=new_fig_laps_context)
new_fig_laps

concatenated_ripple_df.columns: ['P_LR', 'P_RL', 'P_Long', 'P_Short', 'lap_idx', 'lap_start_t', 'P_Long_LR', 'P_Long_RL', 'P_Short_LR', 'P_Short_RL', 'most_likely_decoder_index', 'start', 'stop', 'label', 'duration', 'lap_id', 'lap_dir', 'long_LR_pf_peak_x_pearsonr', 'long_RL_pf_peak_x_pearsonr', 'short_LR_pf_peak_x_pearsonr', 'short_RL_pf_peak_x_pearsonr', 'best_decoder_index', 'session_name', 'time_bin_size', 'delta_aligned_start_t', 'custom_replay_name', 'maze_id', 'is_LR_dir', 'is_most_likely_track_identity_Long', 'is_most_likely_direction_LR', 'epoch_idx', 'long_best_pf_peak_x_pearsonr', 'short_best_pf_peak_x_pearsonr', 'wcorr_long_LR', 'wcorr_long_RL', 'wcorr_short_LR', 'wcorr_short_RL', 'long_best_wcorr', 'short_best_wcorr', 'wcorr_abs_diff', 'pearsonr_abs_diff', 'session_experience_rank', 'session_experience_orientation_rank', 'is_novel_exposure', 'pre_post_delta_category']


,P_LR,P_RL,P_Long,P_Short,lap_idx,lap_start_t,P_Long_LR,P_Long_RL,P_Short_LR,P_Short_RL,most_likely_decoder_index,start,stop,label,duration,lap_id,lap_dir,long_LR_pf_peak_x_pearsonr,long_RL_pf_peak_x_pearsonr,short_LR_pf_peak_x_pearsonr,short_RL_pf_peak_x_pearsonr,best_decoder_index,session_name,time_bin_size,delta_aligned_start_t,custom_replay_name,maze_id,is_LR_dir,is_most_likely_track_identity_Long,is_most_likely_direction_LR,epoch_idx,long_best_pf_peak_x_pearsonr,short_best_pf_peak_x_pearsonr,wcorr_long_LR,wcorr_long_RL,wcorr_short_LR,wcorr_short_RL,long_best_wcorr,short_best_wcorr,wcorr_abs_diff,pearsonr_abs_diff,session_experience_rank,session_experience_orientation_rank,is_novel_exposure,pre_post_delta_category
0,0.466100,0.533900,5.404279e-01,0.459572,0,5.250426,2.518934e-01,2.885345e-01,0.214207,0.245366,1,5.250426,13.692337,0,8.441911,1,1,0.473012,-0.272680,0.351855,-0.544966,3,kdiba_gor01_one_2006-6-07_11-26-53,0.025,-1733.902938,,NaN,NaN,NaN,NaN,0,0.473012,0.351855,0.128830,-0.031509,0.058217,0.008237,0.128830,0.058217,0.070613,0.121157,0.0,0.0,True,pre-delta
1,0.376810,0.623190,5.765410e-01,0.423459,1,29.308670,2.172465e-01,3.592945e-01,0.159564,0.263895,1,29.308670,35.914986,1,6.606316,2,0,-0.477886,0.103281,-0.617921,-0.212387,2,kdiba_gor01_one_2006-6-07_11-26-53,0.025,-1709.844694,,NaN,NaN,NaN,NaN,1,-0.477886,-0.617921,-0.008464,0.013793,0.002362,-0.003495,-0.008464,0.002362,0.006101,-0.140035,0.0,0.0,True,pre-delta
2,0.529244,0.470756,6.404247e-01,0.359575,2,64.076829,3.389411e-01,3.014836e-01,0.190303,0.169272,0,64.076829,68.281973,2,4.205144,3,1,0.677903,-0.296874,0.557521,0.147724,0,kdiba_gor01_one_2006-6-07_11-26-53,0.025,-1675.076535,,NaN,NaN,NaN,NaN,2,-0.296874,0.147724,0.078668,-0.017804,-0.009230,0.014523,-0.017804,0.014523,0.003281,0.149151,0.0,0.0,True,pre-delta
3,0.358672,0.641328,6.488038e-01,0.351196,3,105.519279,2.327077e-01,4.160961e-01,0.125964,0.225232,1,105.519279,109.388817,3,3.869538,4,0,-0.372157,0.388497,0.038310,-0.094784,1,kdiba_gor01_one_2006-6-07_11-26-53,0.025,-1633.634085,,NaN,NaN,NaN,NaN,3,-0.372157,0.038310,-0.069497,0.000453,0.002745,0.006392,-0.069497,0.002745,0.066751,0.333847,0.0,0.0,True,pre-delta
4,0.604020,0.395980,6.741865e-01,0.325814,4,127.074109,4.072221e-01,2.669644e-01,0.196798,0.129016,0,127.074109,134.515034,4,7.440925,5,1,0.683839,-0.093265,0.711045,-0.040914,2,kdiba_gor01_one_2006-6-07_11-26-53,0.025,-1612.079255,,NaN,NaN,NaN,NaN,4,-0.093265,-0.040914,0.085781,0.027607,0.021801,0.001702,0.027607,0.001702,0.025905,0.052351,0.0,0.0,True,pre-delta
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9622,NaN,NaN,NaN,NaN,79,492.297752,NaN,NaN,NaN,NaN,0,492.297752,495.335331,79,3.037579,80,0,0.076337,-0.562835,-0.526973,-0.830214,3,kdiba_vvp01_two_2006-4-28_12-38-13_None,1.500,209.327147,,NaN,NaN,NaN,NaN,79,-0.562835,-0.830214,-0.007060,-0.386929,-0.443352,-0.981208,-0.386929,-0.981208,-0.594279,-0.267380,NaN,NaN,False,post-delta
9623,1.000000,0.000000,1.996995e-46,1.000000,80,497.002526,1.996995e-46,0.000000e+00,1.000000,0.000000,2,497.002526,503.842279,80,6.839753,81,1,0.439159,0.323124,0.640771,0.174394,2,kdiba_vvp01_two_2006-4-28_12-38-13_None,1.500,214.031921,,NaN,NaN,NaN,NaN,80,0.323124,0.174394,0.505080,0.023352,0.943587,0.210732,0.023352,0.210732,-0.187379,0.148730,NaN,NaN,False,post-delta
9624,0.500000,0.500000,4.995963e-01,0.500404,81,507.146030,2.497982e-01,2.497982e-01,0.250202,0.250202,3,507.146030,510.348224,81,3.202194,82,0,0.034472,-0.355392,-0.409640,-0.736704,3,kdiba_vvp01_two_2006-4-28_12-38-13_None,1.500,224.175425,,NaN,NaN,NaN,NaN,81,0.034472,-0.409640,0.064471,-0.449949,-0.216335,-0.989459,0.064471,-0.216335,-0.151864,-0.375169,NaN,NaN,False,post-delta
9625,1.000000,0.000000,3.177038e-24,1.000000,82,511.952190,3.177038e-24,0.000000e+00,1.000000,0.000000,2,511.952190,515.787403,82,3.835213,83,1,0.629688,0.373851,0.83867

'time_bin_size'

num_unique_sessions: 20, num_unique_time_bins: 10
num_unique_sessions: 20, num_unique_time_bins: 10
color_sequence: ['#636EFA', '#EF553B', '#00CC96', '#AB63FA', '#FFA15A', '#19D3F3', '#FF6692', '#B6E880', '#FF97FF', '#FECB52']
len(pre_delta_fig.data): 10
Copied image to clipboard!


Box(children=(Label(value='.html', layout=Layout(width='auto')), HTML(value="<b style='font-size: smaller;'>K:…

Box(children=(Label(value='.png', layout=Layout(width='auto')), HTML(value="<b style='font-size: smaller;'>K:\…

In [ ]:
# variable_name = 'short_best_direction_change_bin_ratio'
# variable_name = 'long_best_congruent_dir_bins_ratio'
variable_name = 'short_best_wcorr'
# 'color':'is_user_annotated_epoch'
# 'color': 'is_user_annotated_epoch', 
px_scatter_kwargs = {'x': 'delta_aligned_start_t', 'y': variable_name, 'title': f"'{variable_name}'"} # , 'color': 'time_bin_size', 'range_y': [-1.0, 1.0], , 'labels': {'session_name': 'Session', 'time_bin_size': 'tbin_size', 'is_user_annotated_epoch':'user_sel'}
# hist_kwargs = dict(color="time_bin_size")
hist_kwargs = dict(color="pre_post_delta_category") # , histnorm='probability density'
new_fig_ripples, figure_context = plotly_pre_post_delta_scatter(data_results_df=concatenated_ripple_df, out_scatter_fig=None, histogram_bins=histogram_bins,
                        px_scatter_kwargs=px_scatter_kwargs, histogram_variable_name=variable_name, hist_kwargs=hist_kwargs, forced_range_y=None,
                        time_delta_tuple=(earliest_delta_aligned_t_start, 0.0, latest_delta_aligned_t_end), is_dark_mode=is_dark_mode)
_extras_output_dict = {}
_extras_output_dict["y_mid_line"] = new_fig_ripples.add_hline(y=0.0, line=dict(color="rgba(0.8,0.8,0.8,.75)", width=2), row='all', col='all')
new_fig_ripples.update_layout(fig_size_kwargs)
new_fig_ripples.show()
figure_out_paths = save_plotly(a_fig=new_fig_ripples, a_fig_context=figure_context)


In [36]:
# variable_name = 'total_congruent_direction_change_diff'
# variable_name = 'long_best_congruent_dir_bins_ratio'
# variable_name = 'long_best_total_congruent_direction_change'
variable_name = 'wcorr_diff'
# variable_name = 'long_best_wcorr'
# 'color':'is_user_annotated_epoch'
# 'color': 'is_user_annotated_epoch', 
px_scatter_kwargs = {'x': 'delta_aligned_start_t', 'y': variable_name, 'color':"is_user_annotated_epoch", 'title': f"'{variable_name}'", 'labels': {'session_name': 'Session', 'time_bin_size': 'tbin_size', 'is_user_annotated_epoch':'user_sel'}} # , 'color': 'time_bin_size', 'range_y': [-1.0, 1.0], 
# hist_kwargs = dict(color="time_bin_size")
hist_kwargs = dict(color="is_user_annotated_epoch") # , histnorm='probability density'
new_fig_ripples, figure_context = plotly_pre_post_delta_scatter(data_results_df=concatenated_ripple_df, out_scatter_fig=None, histogram_bins=histogram_bins,
                        px_scatter_kwargs=px_scatter_kwargs, histogram_variable_name=variable_name, hist_kwargs=hist_kwargs, forced_range_y=None,
                        time_delta_tuple=(earliest_delta_aligned_t_start, 0.0, latest_delta_aligned_t_end), is_dark_mode=is_dark_mode)
_extras_output_dict = {}
_extras_output_dict["y_mid_line"] = new_fig_ripples.add_hline(y=0.0, line=dict(color="rgba(0.8,0.8,0.8,.75)", width=2), row='all', col='all')
new_fig_ripples.update_layout(fig_size_kwargs)
new_fig_ripples.show()

figure_context = figure_context.adding_context_if_missing(num_sessions=num_sessions, plot_type='scatter+hist', comparison='pre-post-delta', variable_name=variable_name)
figure_out_paths = save_plotly(a_fig=new_fig_ripples, a_fig_context=figure_context)


num_unique_sessions: 20, num_unique_time_bins: 10
num_unique_sessions: 20, num_unique_time_bins: 10


ValueError: Value of 'y' is not the name of a column in 'data_frame'. Expected one of ['P_LR', 'P_RL', 'P_Long', 'P_Short', 'lap_idx', 'lap_start_t', 'P_Long_LR', 'P_Long_RL', 'P_Short_LR', 'P_Short_RL', 'most_likely_decoder_index', 'start', 'stop', 'label', 'duration', 'lap_id', 'lap_dir', 'long_LR_pf_peak_x_pearsonr', 'long_RL_pf_peak_x_pearsonr', 'short_LR_pf_peak_x_pearsonr', 'short_RL_pf_peak_x_pearsonr', 'best_decoder_index', 'session_name', 'time_bin_size', 'delta_aligned_start_t', 'custom_replay_name', 'maze_id', 'is_LR_dir', 'is_most_likely_track_identity_Long', 'is_most_likely_direction_LR', 'epoch_idx', 'long_best_pf_peak_x_pearsonr', 'short_best_pf_peak_x_pearsonr', 'wcorr_long_LR', 'wcorr_long_RL', 'wcorr_short_LR', 'wcorr_short_RL', 'long_best_wcorr', 'short_best_wcorr', 'wcorr_abs_diff', 'pearsonr_abs_diff', 'session_experience_rank', 'session_experience_orientation_rank', 'is_novel_exposure', 'pre_post_delta_category'] but received: wcorr_diff

## Stats Tests

In [37]:
from pyphoplacecellanalysis.Pho2D.statistics_plotting_helpers import _perform_stats_tests

stats_variable_name: str = 'P_Short'
# stats_variable_name: str = 'short_best_direction_change_bin_ratio'
# stats_variable_name: str = 'short_best_wcorr'

shuffle_results, p_value, f_value, (dof1, dof2), (variance1, variance2) = _perform_stats_tests(deepcopy(concatenated_ripple_df), stats_variable_name=stats_variable_name)


stats_variable_name: "P_Short" -- actual_diff_means: 0.45242713438454096
stats_variable_name: P_Short
Statistics=237198.50, p=0.00
Reject Null Hypothesis (Significant difference between two samples)
Variance 1: 0.02585915304186724
Variance 2: 0.020403970740396578
Degree of freedom 1: 5118
Degree of freedom 2: 4470
F-statistic: 1.2673588572968437
p-value: 0.9999999999999998


In [38]:
# Show that wcorr in both periods is higher than shuffles

stats_variable_name: str = 'P_Short'
stats_variable_name = 'short_best_wcorr'
# stats_variable_name = 'long_best_wcorr'
# stats_variable_name = 'long_best_wcorr'



shuffle_results, p_value, f_value, (dof1, dof2), (variance1, variance2) = _perform_stats_tests(deepcopy(concatenated_ripple_df), stats_variable_name=stats_variable_name)


# stats_variable_name: "short_best_wcorr" -- actual_diff_means: -0.00983910691641765
# stats_variable_name: short_best_wcorr
# Statistics=72308.00, p=0.73
# Do not Reject Null Hypothesis (No significant difference between two samples)
# Variance 1: 0.1395112660465373
# Variance 2: 0.18436187114204847
# Degree of freedom 1: 395
# Degree of freedom 2: 359
# F-statistic: 0.756725157877388
# p-value: 0.003419223265796241

# stats_variable_name: "long_best_wcorr" -- actual_diff_means: -0.0028337579937901397
# stats_variable_name: long_best_wcorr
# Statistics=71529.00, p=0.93
# Do not Reject Null Hypothesis (No significant difference between two samples)
# Variance 1: 0.1659575407149896
# Variance 2: 0.20687539745971859
# Degree of freedom 1: 395
# Degree of freedom 2: 359
# F-statistic: 0.8022101359215698
# p-value: 0.016221081810852238

stats_variable_name: "short_best_wcorr" -- actual_diff_means: 0.006654626232342094
stats_variable_name: short_best_wcorr
Statistics=11338391.00, p=0.15
Do not Reject Null Hypothesis (No significant difference between two samples)
Variance 1: 0.07452063151809972
Variance 2: 0.08237650320682519
Degree of freedom 1: 5129
Degree of freedom 2: 4496
F-statistic: 0.9046345573931259
p-value: 0.00025791106875214714


In [ ]:
## User non-selected:
scatter_title = f'user_approved_ripple_df Several Sessions {variable_name}'
# variable_name = 'wcorr_abs_diff'
px_scatter_kwargs = {'x': 'delta_aligned_start_t', 'y': variable_name, 'title': scatter_title, 'range_y': [0.0, 1.0], 'labels': {'session_name': 'Session', 'time_bin_size': 'tbin_size'}} # , 'color': 'time_bin_size'
new_fig_ripples, figure_context = plotly_pre_post_delta_scatter(data_results_df=deepcopy(user_approved_ripple_df), out_scatter_fig=None, histogram_bins=histogram_bins,
                        px_scatter_kwargs=px_scatter_kwargs, histogram_variable_name=variable_name, forced_range_y=None,
                        time_delta_tuple=(earliest_delta_aligned_t_start, 0.0, latest_delta_aligned_t_end), is_dark_mode=is_dark_mode)
_extras_output_dict["y_mid_line"] = new_fig_ripples.add_hline(y=0.5, line=dict(color="rgba(0.8,0.8,0.8,.75)", width=2), row='all', col='all')
new_fig_ripples.update_layout(fig_size_kwargs)
new_fig_ripples

In [ ]:
# IDEA: The ones with clear replays (diagonal sequences in the decoded posteriors) are by definiition ambiguous, because there's not much difference between the long/short decoders.


In [ ]:
## User non-selected:
scatter_title = f'Non-selected Several Sessions {variable_name}'
# variable_name = 'wcorr_abs_diff'
px_scatter_kwargs = {'x': 'delta_aligned_start_t', 'y': variable_name, 'title': scatter_title, 'range_y': [0.0, 1.0], 'labels': {'session_name': 'Session', 'time_bin_size': 'tbin_size'}} # , 'color': 'time_bin_size'
new_fig_ripples, figure_context = plotly_pre_post_delta_scatter(data_results_df=deepcopy(user_rejected_ripple_df), out_scatter_fig=None, histogram_bins=histogram_bins,
                        px_scatter_kwargs=px_scatter_kwargs, histogram_variable_name=variable_name, forced_range_y=None,
                        time_delta_tuple=(earliest_delta_aligned_t_start, 0.0, latest_delta_aligned_t_end))
new_fig_ripples.update_layout(fig_size_kwargs)
new_fig_ripples


In [ ]:
# Laps test
concatenated_ripple_df = deepcopy(all_sessions_simple_pearson_laps_df)

scatter_title = 'Several Sessions'
variable_name = 'wcorr_abs_diff'
px_scatter_kwargs = {'x': 'delta_aligned_start_t', 'y': variable_name, 'title': scatter_title, 'range_y': [0.0, 1.0], 'labels': {'session_name': 'Session', 'time_bin_size': 'tbin_size'}} 
new_fig_ripples, figure_context = plotly_pre_post_delta_scatter(data_results_df=deepcopy(concatenated_ripple_df), out_scatter_fig=None, histogram_bins=histogram_bins,
                        px_scatter_kwargs=px_scatter_kwargs, histogram_variable_name=variable_name, forced_range_y=None,
                        time_delta_tuple=(earliest_delta_aligned_t_start, 0.0, latest_delta_aligned_t_end))
new_fig_ripples.update_layout(fig_size_kwargs)
new_fig_ripples


# Plotting functions

In [ ]:
from pyphoplacecellanalysis.General.Model.Configs.LongShortDisplayConfig import PlottingHelpers
# from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import plot_across_sessions_scatter_results

# Example usage:
all_session_figures = plot_across_sessions_scatter_results(collected_outputs_directory, concatenated_laps_df=all_sessions_laps_df, concatenated_ripple_df=all_sessions_ripple_df,
														#    enabled_time_bin_sizes=[0.03, 0.10],
														   earliest_delta_aligned_t_start=earliest_delta_aligned_t_start, latest_delta_aligned_t_end=latest_delta_aligned_t_end,
														   laps_title_prefix=f"Laps", ripple_title_prefix=f"Ripples", save_figures=True, figure_save_extension=['.html','.png'], is_dark_mode=is_dark_mode)
fig_laps, fig_ripples = all_session_figures[0]
# fig_laps.update_layout(fig_size_kwargs)
# fig_ripples.update_layout(fig_size_kwargs)

# fig_laps.show()
fig_ripples.show()
# fig_laps.write_html(f"../output/{TODAY_DAY_DATE}_AcrossSession_fig_laps.html")
# fig_ripples.write_html(f"../output/{TODAY_DAY_DATE}_AcrossSession_fig_ripples.html")


In [ ]:
figure_context = figure_context.adding_context_if_missing(num_sessions=num_sessions, plot_type='scatter+hist', comparison='pre-post-delta', variable_name=variable_name)



In [ ]:
fig_to_clipboard(fig_ripples)

In [ ]:
## time_bin version:
all_time_bin_session_figures = plot_across_sessions_scatter_results(collected_outputs_directory, concatenated_laps_df=all_sessions_laps_time_bin_df, concatenated_ripple_df=all_sessions_ripple_time_bin_df,
														#    enabled_time_bin_sizes=[0.03, 0.10],
														   earliest_delta_aligned_t_start=earliest_delta_aligned_t_start, latest_delta_aligned_t_end=latest_delta_aligned_t_end,
														   main_plot_mode='separate_row_per_session',
														   laps_title_prefix=f"Laps_per_time_bin", ripple_title_prefix=f"Ripples_per_time_bin", save_figures=True, figure_save_extension=['.html','.png'], is_dark_mode=is_dark_mode)
fig_time_bin_laps, fig_time_bin_ripples = all_time_bin_session_figures[0]
# fig_time_bin_laps.show()
fig_time_bin_ripples.show()

In [ ]:
## Test collapsed histograms-only results:
histograms_only_all_time_bin_session_figures = plot_across_sessions_scatter_results(collected_outputs_directory, concatenated_laps_df=all_sessions_laps_time_bin_df, concatenated_ripple_df=all_sessions_ripple_time_bin_df,
														#    enabled_time_bin_sizes=[0.03, 0.10],
															# enabled_time_bin_sizes=[0.03, 0.058, 0.10], # [0.03 , 0.044, 0.058, 0.072, 0.086, 0.1]
														   earliest_delta_aligned_t_start=earliest_delta_aligned_t_start, latest_delta_aligned_t_end=latest_delta_aligned_t_end,
														   main_plot_mode='default',
														   laps_title_prefix=f"Laps_per_time_bin", ripple_title_prefix=f"Ripples_per_time_bin", save_figures=False, figure_save_extension=['.html','.png'])
histograms_only_fig_time_bin_laps, histograms_only_fig_time_bin_ripples = histograms_only_all_time_bin_session_figures[0]
# histograms_only_fig_time_bin_laps.show()
histograms_only_fig_time_bin_ripples.show()

In [ ]:
import plotly.subplots as sp
import plotly.graph_objs as go


all_session_figures = plot_across_sessions_scatter_results(collected_outputs_directory, concatenated_laps_df=single_time_bin_size_all_sessions_laps_df, concatenated_ripple_df=single_time_bin_size_all_sessions_ripple_df,
														#    enabled_time_bin_sizes=[0.03, 0.10],
														   earliest_delta_aligned_t_start=earliest_delta_aligned_t_start, latest_delta_aligned_t_end=latest_delta_aligned_t_end,
														   laps_title_prefix=f"Laps", ripple_title_prefix=f"Ripples", save_figures=True, figure_save_extension=['.html','.png'], is_dark_mode=is_dark_mode, main_plot_mode='default')
fig_laps, fig_ripples = all_session_figures[0]


In [ ]:
from pyphoplacecellanalysis.Pho2D.plotly.Extensions.plotly_helpers import plotly_pre_post_delta_scatter

# data_results_df: pd.DataFrame = deepcopy(all_sessions_laps_df) # all_sessions_laps_time_bin_df
data_results_df: pd.DataFrame = deepcopy(single_time_bin_size_all_sessions_laps_time_bin_df).infer_objects()

# histogram_bins = 'auto'
histogram_bins: int = 25
new_laps_fig, new_laps_fig_context = plotly_pre_post_delta_scatter(data_results_df=data_results_df, out_scatter_fig=fig_laps, histogram_bins=histogram_bins, px_scatter_kwargs=dict(title='Laps'))


# new_laps_fig = new_laps_fig.update_layout(fig_size_kwargs, 
#     xaxis_title="X Axis Title",
#     yaxis_title="Y Axis Title",
#     legend_title="Legend Title",
#     font=dict(
#         family="Courier New, monospace",
#         size=18,
#         color="RebeccaPurple"
#     ),
# )
# Update x-axis labels
# new_laps_fig.update_xaxes(title_text="Num Time Bins", row=1, col=1)
# new_laps_fig.update_xaxes(title_text="Delta-aligned Event Time (seconds)", row=1, col=2)
# new_laps_fig.update_xaxes(title_text="Num Time Bins", row=1, col=3)


_extras_output_dict = {}
_extras_output_dict["y_mid_line"] = new_laps_fig.add_hline(y=0.5, line=dict(color="rgba(0.8,0.8,0.8,.75)", width=2), row='all', col='all')

new_laps_fig

# # Update layout to add a title to the legend
# new_fig_ripples.update_layout(
#     legend_title_text='Is User Selected'  # Add a title to the legend
# )

# fig_to_clipboard(new_fig_ripples, **fig_size_kwargs)

new_laps_fig_context: IdentifyingContext = new_laps_fig_context.adding_context_if_missing(epoch='laps', num_sessions=num_sessions, plot_type='scatter+hist', comparison='pre-post-delta', variable_name=variable_name)
figure_out_paths = save_plotly(a_fig=new_laps_fig, a_fig_context=new_laps_fig_context)
new_laps_fig

In [ ]:
data_results_df


In [ ]:
np.unique(data_results_df['time_bin_size'])


In [ ]:

data_results_df[data_results_df['time_bin_size'] == 0.005]

# Matplotlib-based versions:

In [ ]:
from pyphoplacecellanalysis.Pho2D.statistics_plotting_helpers import plot_histograms_across_sessions, plot_stacked_histograms

matplotlib.use('Qt5Agg')


In [ ]:

# #TODO 2024-06-05 09:15: - [ ] Define save_matplotlib function
new_laps_fig_context: IdentifyingContext = new_laps_fig_context.adding_context_if_missing(epoch='laps', num_sessions=num_sessions, plot_type='scatter+hist', comparison='pre-post-delta', variable_name=variable_name)

In [ ]:
# from Spike3D.PendingNotebookCode import plot_stacked_histograms

# You can use it like this:'long_best_dir_simple_pearsonr'
_out0: "MatplotlibRenderPlots" = plot_histograms_across_sessions(data_type='Laps', session_spec='All Sessions', data_results_df=all_sessions_laps_time_bin_df, time_bin_duration_str="75 ms")
_out1: "MatplotlibRenderPlots" = plot_histograms_across_sessions(data_type='Ripples', session_spec='All Sessions', data_results_df=all_sessions_ripple_time_bin_df, time_bin_duration_str="75 ms")

In [ ]:


fig_to_clipboard(_out0.figures[0])

In [ ]:
## all_sessions_new_laps_df, all_sessions_new_ripple_df
_out0: "MatplotlibRenderPlots" = plot_histograms_across_sessions(data_type='New Laps', session_spec='All Sessions', data_results_df=all_sessions_new_laps_df, time_bin_duration_str="25 ms")
_out1: "MatplotlibRenderPlots" = plot_histograms_across_sessions(data_type='New Ripples', session_spec='All Sessions', data_results_df=all_sessions_new_ripple_df, time_bin_duration_str="25 ms")



In [ ]:
all_sessions_new_laps_df

In [ ]:
len(all_sessions_new_laps_df['session_name'].unique()) # 10 sessions



In [ ]:

matplotlib.use('Qt5Agg')

column_name: str = 'P_Short'

# You can use it like this:
num_unique_sessions: int = all_sessions_laps_time_bin_df.session_name.nunique(dropna=True) # number of unique sessions, ignoring the NA entries
num_unique_time_bins: int = all_sessions_laps_time_bin_df.time_bin_size.nunique(dropna=True)
_laps_histogram_out = plot_stacked_histograms(all_sessions_laps_time_bin_df, 'Laps', f'{num_unique_sessions} Sessions', f"{num_unique_time_bins} tbin sizes", column_name=column_name)

fig_to_clipboard(_laps_histogram_out.figures[0])

num_unique_sessions: int = all_sessions_ripple_time_bin_df.session_name.nunique(dropna=True) # number of unique sessions, ignoring the NA entries
num_unique_time_bins: int = all_sessions_ripple_time_bin_df.time_bin_size.nunique(dropna=True)
_ripple_histogram_out = plot_stacked_histograms(all_sessions_ripple_time_bin_df, 'Ripples', f'{num_unique_sessions} Sessions', f"{num_unique_time_bins} tbin sizes", column_name=column_name)



In [ ]:
all_sessions_ripple_time_bin_df['time_bin_size'].unique()

In [ ]:
# active_time_bin_size: float = 0.072 # 0.058
# active_time_bin_size: float = 0.086
# active_time_bin_size: float = 0.1 # looks the most bimodal with as little of an intermediate value as possible.
active_time_bin_size: float = 0.13444444 # looks the most bimodal with as little of an intermediate value as possible.
_single_time_bin_ripple_histogram_out = plot_stacked_histograms(all_sessions_ripple_time_bin_df[np.isclose(all_sessions_ripple_time_bin_df['time_bin_size'], active_time_bin_size)], 'Ripples', f'{num_unique_sessions} Sessions', f"tbin: {active_time_bin_size:0.3f}s", column_name=column_name)
fig_to_clipboard(_single_time_bin_ripple_histogram_out.figures[0], bbox_inches='tight')


In [ ]:
fig_to_clipboard(_ripple_histogram_out.figures[0])